# Create the intermediate layer for the US Climate warehouse
### Declare variables used throughout

In [ ]:
project_id = "kiaraerica"
region = "us-central1"
model_name = "gemini-1.5-flash"
dataset = "us_climate_int"

### Create BQ dataset for storing the intermediate data

In [ ]:
from google.cloud import bigquery

bq_client = bigquery.Client()

dataset_id = bigquery.Dataset(f"{project_id}.{dataset}")
dataset_id.location = region
resp = bq_client.create_dataset(dataset_id, exists_ok=True)
print("Created dataset {}.{}".format(bq_client.project, resp.dataset_id))

Created dataset kiaraerica.us_climate_int


## Create `Organizations` table (Criteria 9)

### Create table that stores facilities with no organization listed

Create `tmp_ccf_facilities_no_org` table that stores all the carbon capture facilities that has no organization listed in the `carbon_capture_facilities` table

In [ ]:
%%bigquery
create or replace table us_climate_int.tmp_ccf_facilities_no_org as
select distinct
    id as ccf_id,
    facility as ccf_facility_name,
    organization,
    city,
    state,
    category,
    status,
    industry,
    _data_source,
    _load_time
from us_climate_stg.carbon_capture_facilities
where facility is not null and organization is null
order by ccf_facility_name;

Query is running:   0%|          |

""


Create `tmp_ghg_facilities_no_org` table that stores all the ghg emission facilities that has no organization listed in the `facility_ghg_emissions` table

In [ ]:
%%bigquery
create or replace table us_climate_int.tmp_ghg_facilities_no_org as
WITH ranked as (
  select
      facility_id,
      facility_name,
      city,
      state,
      naics_code,
      industry_sector1,
      industry_sector2,
      industry_sector3,
      max_rated_heat_input_capacity,
      carbon_dioxide_emissions,
      methane_emissions,
      nitrous_oxide_emissions,
      biogenic_co2_emissions,
      _data_source,
      _load_time,
      row_number() over (partition by facility_id order by _load_time desc) as rk
  from us_climate_stg.facility_ghg_emissions
  where facility_name is not null
)
select *
from ranked
where rk = 1
order by facility_name;

Query is running:   0%|          |

""


### Use LLM to find their organizations

Use LLM to find which organization does the ghg emission facility belongs to

In [ ]:
import json
import pandas as pd
import pandas_gbq
from google.cloud import bigquery
import vertexai
from vertexai.generative_models import GenerativeModel

vertexai.init(project=project_id, location=region)
model = GenerativeModel(model_name=model_name)
bq_client = bigquery.Client(project=project_id)

sql_ccf = """
SELECT
  ccf_id,
  ccf_facility_name,
  city,
  state,
  category,
  status,
  industry
FROM us_climate_int.tmp_ccf_facilities_no_org
"""
rows_ccf = list(bq_client.query(sql_ccf).result())
df_ccf = bq_client.query(sql_ccf).to_dataframe()

prompt_ccf = """Given a carbon capture facility record with:
ccf_facility_name, city, state, category, status, industry.

Identify the organization that owns or operates this facility, or return null if unknown.
Return EXACTLY one JSON line:
{
  "ccf_id": <long>,
  "organization_name": <string or null>
}
No extra text or explanation.
"""

def find_org_for_ccf(row):
    row_dict = row.to_dict()  # Convert row to a dictionary

    row_text = (
        f"ccf_id={row_dict.get('ccf_id', 'Unknown')}, "
        f"ccf_facility_name='{row_dict.get('ccf_facility_name', 'Unknown')}', "
        f"city='{row_dict.get('city', 'Unknown')}', "
        f"state='{row_dict.get('state', 'Unknown')}', "
        f"category='{row_dict.get('category', 'Unknown')}', "
        f"status='{row_dict.get('status', 'Unknown')}', "
        f"industry='{row_dict.get('industry', 'Unknown')}'"
    )
    combined_prompt = row_text + "\n" + prompt_ccf

    resp = model.generate_content(combined_prompt)
    raw_text = resp.text.replace("```json", "").replace("```", "").strip()

    try:
        return json.loads(raw_text)
    except:
        return {"ccf_id": row_dict.get("ccf_id", None), "organization_name": None}


results_ccf = []
for _, ccf_row in df_ccf.iterrows():
    out = find_org_for_ccf(ccf_row)
    results_ccf.append(out)

df_ccf_out = pd.DataFrame(results_ccf)
pandas_gbq.to_gbq(
    df_ccf_out,
    "us_climate_int.tmp_ccf_facilities_llm_org",
    project_id=project_id,
    if_exists="replace"
)
print("LLM results for CCF saved to tmp_ccf_facilities_llm_org.")

100%|██████████| 1/1 [00:00<00:00, 7489.83it/s]

LLM results for CCF saved to tmp_ccf_facilities_llm_org.


Use LLM to find which organization does the carbon capture facility belongs to

In [ ]:
import json
import time
import pandas as pd
import pandas_gbq
from google.cloud import bigquery
import vertexai
from vertexai.generative_models import GenerativeModel
from google.api_core.exceptions import GoogleAPIError

vertexai.init(project=project_id, location=region)
model = GenerativeModel(model_name=model_name)
bq_client = bigquery.Client(project=project_id)

sql_create_checkpoint = """
CREATE TABLE IF NOT EXISTS us_climate_int.tmp_ghg_facilities_llm_org_checkpoint (
    facility_id STRING,
    organization_name STRING
);
"""
bq_client.query(sql_create_checkpoint).result()
print("Checkpoint table is ready.")

sql_ghg = """
WITH all_facilities AS (
    SELECT
        CAST(facility_id AS STRING) AS facility_id,
        facility_name,
        city,
        state,
        naics_code,
        industry_sector1,
        industry_sector2,
        industry_sector3
    FROM us_climate_int.tmp_ghg_facilities_no_org
    ORDER BY facility_id
),
unprocessed_facilities AS (
    SELECT a.*
    FROM all_facilities a
    LEFT JOIN us_climate_int.tmp_ghg_facilities_llm_org_checkpoint c
    ON a.facility_id = c.facility_id
    WHERE c.facility_id IS NULL
)
SELECT * FROM unprocessed_facilities
"""
df_ghg = bq_client.query(sql_ghg).to_dataframe()
print(f"Found {len(df_ghg)} unprocessed facilities.")

prompt_ghg = """Given a facility from the GHG emissions dataset:
facility_id, facility_name, city, state, naics_code, industry_sector1,2,3.

Identify the organization that owns or operates this facility, or return null if unknown.
Return EXACTLY one JSON line:
{
  "facility_id": <string>,
  "organization_name": <string or null>
}
No extra text or explanation.
"""

def find_org_for_ghg(row):
    row_dict = row.to_dict()

    row_text = (
        f"facility_id={row_dict.get('facility_id', 'Unknown')}, "
        f"facility_name='{row_dict.get('facility_name', 'Unknown')}', "
        f"city='{row_dict.get('city', 'Unknown')}', "
        f"state='{row_dict.get('state', 'Unknown')}', "
        f"naics_code='{row_dict.get('naics_code', 'Unknown')}', "
        f"sector1='{row_dict.get('industry_sector1', 'Unknown')}', "
        f"sector2='{row_dict.get('industry_sector2', 'Unknown')}', "
        f"sector3='{row_dict.get('industry_sector3', 'Unknown')}'"
    )

    combined_prompt = row_text + "\n" + prompt_ghg

    try:
        resp = model.generate_content(combined_prompt)
        raw_text = resp.text.replace("```json", "").replace("```", "").strip()
        parsed = json.loads(raw_text)

        if "facility_id" not in parsed:
            parsed["facility_id"] = row_dict.get("facility_id", None)
        return parsed

    except Exception as e:
        print(f"Error processing facility_id {row_dict.get('facility_id', None)}: {e}")
        return {
            "facility_id": row_dict.get("facility_id", None),
            "organization_name": None
        }

batch_size = 50
min_batch_size = 5
sleep_time = 5
max_retries = 5

results_ghg = []
i = 0

while i < len(df_ghg):
    try:
        batch_df = df_ghg.iloc[i:i + batch_size]
        batch_results = [find_org_for_ghg(row) for _, row in batch_df.iterrows()]

        df_batch = pd.DataFrame(batch_results)
        pandas_gbq.to_gbq(
            df_batch,
            "us_climate_int.tmp_ghg_facilities_llm_org_checkpoint",
            project_id=project_id,
            if_exists="append"
        )

        results_ghg.extend(batch_results)
        i += batch_size
        print(f"Processed {i}/{len(df_ghg)} records.")

        time.sleep(sleep_time)

    except GoogleAPIError as e:
        print(f"Quota error encountered: {e}. Retrying with backoff...")

        for retry in range(1, max_retries + 1):
            time.sleep(sleep_time * retry)
            print(f"Retrying (attempt {retry}/{max_retries})...")
            try:
                batch_df = df_ghg.iloc[i:i + batch_size]
                batch_results = [find_org_for_ghg(row) for _, row in batch_df.iterrows()]

                df_batch = pd.DataFrame(batch_results)
                pandas_gbq.to_gbq(
                    df_batch,
                    "us_climate_int.tmp_ghg_facilities_llm_org_checkpoint",
                    project_id=project_id,
                    if_exists="append"
                )

                results_ghg.extend(batch_results)
                i += batch_size
                print(f"Processed {i}/{len(df_ghg)} records after retry.")

                break

            except GoogleAPIError:
                if retry == max_retries:
                    print("Max retries reached. Reducing batch size.")
                    batch_size = max(batch_size // 2, min_batch_size)
                    if batch_size == min_batch_size:
                        print("Minimum batch size reached. Exiting.")
                        break

df_ghg_out = pd.DataFrame(results_ghg)
pandas_gbq.to_gbq(
    df_ghg_out,
    "us_climate_int.tmp_ghg_facilities_llm_org",
    project_id=project_id,
    if_exists="replace"
)

print("LLM results for GHG saved to tmp_ghg_facilities_llm_org.")

Checkpoint table is ready.
Found 581 unprocessed facilities.


100%|██████████| 1/1 [00:00<00:00, 6141.00it/s]


Processed 50/581 records.


100%|██████████| 1/1 [00:00<00:00, 10034.22it/s]


Processed 100/581 records.


...

Processed 550/581 records.


100%|██████████| 1/1 [00:00<00:00, 8322.03it/s]


Processed 600/581 records.


100%|██████████| 1/1 [00:00<00:00, 8542.37it/s]

LLM results for GHG saved to tmp_ghg_facilities_llm_org.


### Normalize the organization names

Get all distinct organization names in the ghg emissions facility table



In [ ]:
%%bigquery
SELECT DISTINCT organization_name
FROM us_climate_int.tmp_ghg_facilities_llm_org_checkpoint
WHERE organization_name IS NOT NULL
ORDER BY organization_name;

Query is running:   0%|          |

Downloading:   0%|          |

,organization_name
0,3M
1,3M Company
2,"ABE South Dakota, LLC"
3,ABENGOA BIOENERGY CORPORATION
4,ABERDEEN ENERGY LLC
...,...
2169,Yale University
2170,"Yara West Sacramento Terminal, LLC"
2171,York Plant Holding LLC
2172,null


Use LLM to normalize the carbon capture facilities' organizations' names

In [ ]:
import json
import time
from google.cloud import bigquery
import vertexai
from vertexai.generative_models import GenerativeModel

# Initialize BigQuery client
bq_client = bigquery.Client(project=project_id)

# Query organizations from both the checkpoint table (missing ones) and the staging table (existing ones)
sql_orgs = """
SELECT DISTINCT organization_name
FROM us_climate_int.tmp_ccf_facilities_llm_org_checkpoint
WHERE organization_name IS NOT NULL

UNION DISTINCT

SELECT DISTINCT organization
FROM us_climate_stg.carbon_capture_facilities
WHERE organization IS NOT NULL
"""
query_job = bq_client.query(sql_orgs)
org_names = [row["organization_name"] for row in query_job]

# Initialize Vertex AI model
vertexai.init(project=project_id, location=region)
model = GenerativeModel(model_name=model_name)

org_mapping = []

# Function to get standardized organization name and country
def get_standardized_name(org, retries=3):
    prompt = f"""
    Normalize the organization name and determine the country this organization belongs to.
    For example, X and X LLC should be the same company X, Z glass and Z group should be the same company.
    No abbreviations like "US" or "UK". Convert organizations like "N/A", "None", "null" all to null.

    Given the organization: "{org}"

    Return JSON in this format:
    {{
      "original_name": "{org}",
      "standardized_name": "<Standardized Organization Name>",
      "country": "<Country Name>"
    }}

    No extra text or explanation. Only return valid JSON.
    """

    for attempt in range(retries):
        try:
            response = model.generate_content(prompt)

            if response and response.text.strip():
                response_text = response.text.strip().replace("```json", "").replace("```", "").strip()
                parsed_data = json.loads(response_text)

                if "original_name" in parsed_data and "standardized_name" in parsed_data and "country" in parsed_data:
                    return parsed_data

            print(f"Warning: Empty or invalid response for {org}. Retrying ({attempt + 1}/{retries})...")

        except json.JSONDecodeError as e:
            print(f"Error parsing JSON for {org}: {e}. Retrying ({attempt + 1}/{retries})...")
        except Exception as e:
            print(f"Unexpected error with LLM for {org}: {e}. Retrying ({attempt + 1}/{retries})...")

        time.sleep(50)

    print(f"Failed to get standardized name for {org} after {retries} attempts.")
    return {"original_name": org, "standardized_name": org, "country": "Unknown"}

# Process all organizations and get their standardized names
for org in org_names:
    org_mapping.append(get_standardized_name(org))

print("Final standardized organization mapping generated.")

# Load the standardized names into the BigQuery table
table_id = "us_climate_int.tmp_ccf_organization_name_mapping"

schema = [
    bigquery.SchemaField("original_name", "STRING"),
    bigquery.SchemaField("standardized_name", "STRING"),
    bigquery.SchemaField("country", "STRING"),
]

job_config = bigquery.LoadJobConfig(schema=schema, write_disposition="WRITE_TRUNCATE")
job = bq_client.load_table_from_json(org_mapping, table_id, job_config=job_config)
job.result()
print(f"Organization name mapping table saved in {table_id}.")

Final standardized organization mapping generated.
Organization name mapping table saved in us_climate_int.tmp_ccf_organization_name_mapping.


Use LLM to normalize the carbon emission facilities' organizations' names

In [ ]:
import json
import time
from google.cloud import bigquery
import vertexai
from vertexai.generative_models import GenerativeModel

bq_client = bigquery.Client(project=project_id)

sql_orgs = """
SELECT DISTINCT organization_name
FROM us_climate_int.tmp_ghg_facilities_llm_org_checkpoint
WHERE organization_name IS NOT NULL
"""
query_job = bq_client.query(sql_orgs)
org_names = [row["organization_name"] for row in query_job]

vertexai.init(project=project_id, location=region)
model = GenerativeModel(model_name=model_name)

org_mapping = []

def get_standardized_name(org, retries=3):
    prompt = f"""
    Normalize the organization name and determine the country this organization belongs to. For example, X and X LLC should be the same company X, Z glass and Z group should be the same company. No abbreviations like "US" or "UK". Convert organizations like "N/A", "None", "null" all to null.

    Given the organization: "{org}"

    Return JSON in this format:
    {{
      "original_name": "{org}",
      "standardized_name": "<Standardized Organization Name>",
      "country": "<Country Name>"
    }}

    No extra text or explanation. Only return valid JSON.
    """

    for attempt in range(retries):
        try:
            response = model.generate_content(prompt)

            if response and response.text.strip():
                response_text = response.text.strip().replace("```json", "").replace("```", "").strip()
                parsed_data = json.loads(response_text)

                if "original_name" in parsed_data and "standardized_name" in parsed_data and "country" in parsed_data:
                    return parsed_data

            print(f"Warning: Empty or invalid response for {org}. Retrying ({attempt + 1}/{retries})...")

        except json.JSONDecodeError as e:
            print(f"Error parsing JSON for {org}: {e}. Retrying ({attempt + 1}/{retries})...")
        except Exception as e:
            print(f"Unexpected error with LLM for {org}: {e}. Retrying ({attempt + 1}/{retries})...")

        time.sleep(50)

    print(f"Failed to get standardized name for {org} after {retries} attempts.")
    return {"original_name": org, "standardized_name": org, "country": "Unknown"}

for org in org_names:
    org_mapping.append(get_standardized_name(org))

print("Final standardized organization mapping generated.")

table_id = "us_climate_int.tmp_ghg_organization_name_mapping"

schema = [
    bigquery.SchemaField("original_name", "STRING"),
    bigquery.SchemaField("standardized_name", "STRING"),
    bigquery.SchemaField("country", "STRING"),
]

job_config = bigquery.LoadJobConfig(schema=schema, write_disposition="WRITE_TRUNCATE")
job = bq_client.load_table_from_json(org_mapping, table_id, job_config=job_config)
job.result()
print(f"Organization name mapping table saved in {table_id}.")

Final standardized organization mapping generated.
Organization name mapping table saved in us_climate_int.tmp_ghg_organization_name_mapping.


Create `us_climate_int.Organization` table that stores all organizations that have the carbon emission or the carbon capture facilities

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE us_climate_int.tmp_Organization AS
WITH combined_data AS (
    SELECT
        standardized_name AS organization_name,
        country,
        (SELECT DISTINCT _data_source FROM us_climate_stg.carbon_capture_facilities LIMIT 1) AS _data_source,
        (SELECT DISTINCT _load_time FROM us_climate_stg.carbon_capture_facilities LIMIT 1) AS _load_time
    FROM us_climate_int.tmp_ccf_organization_name_mapping

    UNION ALL

    SELECT
        standardized_name AS organization_name,
        country,
        (SELECT DISTINCT _data_source FROM us_climate_stg.facility_ghg_emissions LIMIT 1) AS _data_source,
        (SELECT DISTINCT _load_time FROM us_climate_stg.facility_ghg_emissions LIMIT 1) AS _load_time
    FROM us_climate_int.tmp_ghg_organization_name_mapping
),

deduplicated AS (
    SELECT
        NULLIF(TRIM(organization_name), '') AS organization_name,
        NULLIF(TRIM(MAX(country)), '') AS country,
        MAX(_data_source) AS _data_source,
        MAX(_load_time) AS _load_time
    FROM combined_data
    GROUP BY organization_name
)

SELECT * FROM deduplicated;

Query is running:   0%|          |

""


### Assign Primary Key

Check the uniqueness of organization_name in the `Organization` table

In [ ]:
%%bigquery
SELECT organization_name, COUNT(*) AS count
FROM us_climate_int.tmp_Organization
GROUP BY organization_name
HAVING COUNT(*) > 1
ORDER BY count DESC;

Query is running:   0%|          |

Downloading: |          |

,organization_name,count


Create final Organization table

In [ ]:
%%bigquery
create or replace table us_climate_int.Organization as
  select *
  from us_climate_int.tmp_Organization

Query is running:   0%|          |

""


In [ ]:
%%bigquery
create or replace table us_climate_int.Organizations as
  select *
  from us_climate_int.Organization

Query is running:   0%|          |

""


Clean up

In [ ]:
%%bigquery
drop table if exists us_climate_int.tmp_Organization;
drop table if exists us_climate_int.tmp_ccf_facilities_llm_org;
drop table if exists us_climate_int.tmp_ccf_facilities_llm_org_checkpoint;
drop table if exists us_climate_int.tmp_ccf_facilities_no_org;
drop table if exists us_climate_int.tmp_ccf_organization_name_mapping;
drop table if exists us_climate_int.tmp_ghg_facilities_llm_org;
drop table if exists us_climate_int.tmp_ghg_facilities_llm_org_checkpoint;
drop table if exists us_climate_int.tmp_ghg_facilities_no_org;
drop table if exists us_climate_int.tmp_ghg_organization_name_mapping;
drop table if exists us_climate_int.Organization

Query is running:   0%|          |

""


## Create `Geo_References` table (criteria 9)

This table will include the abbreviation for state/province/territories we have data for as the id, as well as the full name spelled out and the capital. It will be used in other tables when referencing state/province/territories.

`state_disasters` table has the abbreviation for all 50 states, using it as a base.

In [ ]:
%%bigquery
select distinct state as geo_id
from us_climate_stg.state_disasters

Query is running:   0%|          |

Downloading:   0%|          |

,geo_id
0,NY
1,CT
2,VT
3,WV
4,PA
5,HI
6,NV
7,RI
8,IN
9,MI


In [ ]:
%%bigquery
create or replace table us_climate_int.tmp_geo_reference as
    select distinct state as geo_id
    from us_climate_stg.state_disasters

Query is running:   0%|          |

""


`state_ghg_emissions` includes data for U.S. Territories

In [ ]:
%%bigquery
select distinct geo_ref
from us_climate_stg.state_ghg_emissions
where geo_ref not in (select geo_id from us_climate_int.tmp_geo_reference)

Query is running:   0%|          |

Downloading:   0%|          |

,geo_ref
0,DC
1,AS
2,GU
3,UM
4,VI
5,MP
6,PR
7,MH
8,FM
9,PW


`carbon_capture_facilities` includes data for some Canadian provinces

In [ ]:
%%bigquery
select distinct state
from us_climate_stg.carbon_capture_facilities
where state not in (select geo_id from us_climate_int.tmp_geo_reference)

Query is running:   0%|          |

Downloading:   0%|          |

,state
0,AB
1,SK
2,BC


In [ ]:
%%bigquery
insert into us_climate_int.tmp_geo_reference (geo_id)
select distinct geo_ref
from us_climate_stg.state_ghg_emissions
where geo_ref not in (select geo_id from us_climate_int.tmp_geo_reference)

union distinct

select distinct state
from us_climate_stg.carbon_capture_facilities
where state not in (select geo_id from us_climate_int.tmp_geo_reference)

Query is running:   0%|          |

""


Adding columns `name` and `capital`

In [ ]:
%%bigquery
alter table us_climate_int.tmp_geo_reference
add column name string,
add column capital string

Query is running:   0%|          |

""


Use LLM to match abbreviation to name and capital

In [ ]:
import json
import pandas as pd
import pandas_gbq
from google.cloud import bigquery
import vertexai
from vertexai.generative_models import GenerativeModel

prompt = """For each place abbreviation provided, return the full name and its capital.
Focus only on U.S. states, U.S. districts, U.S. territories, and Canadian provinces.
If you can't find name, return null.
If you can't find capital, return null.
Format the answer as a list of dictionaries with the schema:
{"geo_id": "<abbreviation>", "name": "<full name>", "capital": "<capital>"}
Example:
[{"geo_id": "TX", "full_name": "Texas", "capital": "Austin"},
 {"geo_id": "ON", "full_name": "Ontario", "capital": "Toronto"}
"""

bq_client = bigquery.Client()

input_sql = """select geo_id from us_climate_int.tmp_geo_reference
where name is null or capital is null
"""

input_str = ""

rows = bq_client.query_and_wait(input_sql)

for row in rows:
    input_str += f"{row['geo_id']}, "
input_str = input_str[:-2]

vertexai.init(project=project_id, location=region)
model = GenerativeModel(model_name)
resp = model.generate_content([input_str, prompt])
resp_text = resp.text.replace("```json", "").replace("```", "").replace("\n", "")
print("results_raw:", resp_text)
replacements = json.loads(resp_text)
print("replacements:", replacements)

df = pd.DataFrame(replacements)

table_id = "us_climate_int.tmp_geo_reference"
pandas_gbq.to_gbq(df, table_id, project_id=project_id, if_exists="replace")

results_raw: [  {"geo_id": "LA", "name": "Louisiana", "capital": "Baton Rouge"},  {"geo_id": "FL", "name": "Florida", "capital": "Tallahassee"},  {"geo_id": "DE", "name": "Delaware", "capital": "Dover"},  {"geo_id": "MS", "name": "Mississippi", "capital": "Jackson"},  {"geo_id": "ID", "name": "Idaho", "capital": "Boise"},  {"geo_id": "AK", "name": "Alaska", "capital": "Juneau"},  {"geo_id": "NM", "name": "New Mexico", "capital": "Santa Fe"},  {"geo_id": "OK", "name": "Oklahoma", "capital": "Oklahoma City"},  {"geo_id": "IL", "name": "Illinois", "capital": "Springfield"},  {"geo_id": "IA", "name": "Iowa", "capital": "Des Moines"},  {"geo_id": "ME", "name": "Maine", "capital": "Augusta"},  {"geo_id": "IN", "name": "Indiana", "capital": "Indianapolis"},  {"geo_id": "WV", "name": "West Virginia", "capital": "Charleston"},  {"geo_id": "MD", "name": "Maryland", "capital": "Annapolis"},  {"geo_id": "NY", "name": "New York", "capital": "Albany"},  {"geo_id": "AZ", "name": "Arizona", "capital":

100%|██████████| 1/1 [00:00<00:00, 8525.01it/s]


In [ ]:
%%bigquery
select * from us_climate_int.tmp_geo_reference

Query is running:   0%|          |

Downloading:   0%|          |

,geo_id,name,capital
0,NY,New York,Albany
1,MD,Maryland,Annapolis
2,GA,Georgia,Atlanta
3,ME,Maine,Augusta
4,TX,Texas,Austin
...,...,...,...
61,DC,District of Columbia,"Washington, D.C."
62,FO,null,null
63,UM,null,null
64,National,null,null


`null` is stored as string

In [ ]:
%%bigquery
select * from us_climate_int.tmp_geo_reference where name = 'null'

Query is running:   0%|          |

Downloading:   0%|          |

,geo_id,name,capital
0,FO,null,null
1,UM,null,null
2,National,null,null
3,USTERR,null,null


Converting 'null' to `null`

In [ ]:
%%bigquery
update us_climate_int.tmp_geo_reference
set name = null
where name = 'null';

update us_climate_int.tmp_geo_reference
set capital = null
where capital = 'null';

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select * from us_climate_int.tmp_geo_reference where name is null

Query is running:   0%|          |

Downloading:   0%|          |

,geo_id,name,capital
0,USTERR,None,None
1,National,None,None
2,FO,None,None
3,UM,None,None


Adding `_data_source` and `_load_time`

In [ ]:
%%bigquery
select distinct
    geo_ref.geo_id,
    geo_ref.name,
    geo_ref.capital,
    sd._data_source,
    sd._load_time
from us_climate_int.tmp_geo_reference geo_ref
left join us_climate_stg.state_disasters sd
  on geo_ref.geo_id = sd.state

Query is running:   0%|          |

Downloading:   0%|          |

,geo_id,name,capital,_data_source,_load_time
0,FO,None,None,None,NaT
1,GU,Guam,Hagåtña,None,NaT
2,AS,American Samoa,Pago Pago,None,NaT
3,AB,Alberta,Edmonton,None,NaT
4,VI,Virgin Islands,Charlotte Amalie,None,NaT
...,...,...,...,...,...
61,MO,Missouri,Jefferson City,noaa,2025-02-04 16:02:38.600918+00:00
62,WA,Washington,Olympia,noaa,2025-02-04 16:02:38.600918+00:00
63,NV,Nevada,Carson City,noaa,2025-02-04 16:02:38.600918+00:00
64,FL,Florida,Tallahassee,noaa,2025-02-04 16:02:38.600918+00:00


In [ ]:
%%bigquery
create or replace table us_climate_int.tmp_geo_reference as
  select distinct
      geo_ref.geo_id,
      geo_ref.name,
      geo_ref.capital,
      sd._data_source,
      sd._load_time
  from us_climate_int.tmp_geo_reference geo_ref
  left join us_climate_stg.state_disasters sd
    on geo_ref.geo_id = sd.state

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select distinct
    geo_ref.geo_id,
    geo_ref.name,
    geo_ref.capital,
    coalesce(geo_ref._data_source, ghg._data_source, cc._data_source) as _data_source,
    coalesce(geo_ref._load_time, ghg._load_time, cc._load_time) as _load_time
from us_climate_int.tmp_geo_reference geo_ref
left join us_climate_stg.state_ghg_emissions ghg
  on geo_ref.geo_id = ghg.geo_ref
left join us_climate_stg.carbon_capture_facilities cc
  on geo_ref.geo_id = cc.state

Query is running:   0%|          |

Downloading:   0%|          |

,geo_id,name,capital,_data_source,_load_time
0,FO,None,None,usepa,2025-02-06 21:30:29.918903+00:00
1,GU,Guam,Hagåtña,usepa,2025-02-06 21:30:29.918903+00:00
2,AS,American Samoa,Pago Pago,usepa,2025-02-06 21:30:29.918903+00:00
3,AB,Alberta,Edmonton,bbi-international,2025-02-04 16:21:17.573644+00:00
4,VI,Virgin Islands,Charlotte Amalie,usepa,2025-02-06 21:30:29.918903+00:00
...,...,...,...,...,...
61,MO,Missouri,Jefferson City,noaa,2025-02-04 16:02:38.600918+00:00
62,WA,Washington,Olympia,noaa,2025-02-04 16:02:38.600918+00:00
63,NV,Nevada,Carson City,noaa,2025-02-04 16:02:38.600918+00:00
64,FL,Florida,Tallahassee,noaa,2025-02-04 16:02:38.600918+00:00


In [ ]:
%%bigquery
create or replace table us_climate_int.tmp_geo_reference as
  select distinct
      geo_ref.geo_id,
      geo_ref.name,
      geo_ref.capital,
      coalesce(geo_ref._data_source, ghg._data_source, cc._data_source) as _data_source,
      coalesce(geo_ref._load_time, ghg._load_time, cc._load_time) as _load_time
  from us_climate_int.tmp_geo_reference geo_ref
  left join us_climate_stg.state_ghg_emissions ghg
    on geo_ref.geo_id = ghg.geo_ref
  left join us_climate_stg.carbon_capture_facilities cc
    on geo_ref.geo_id = cc.state

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select * from us_climate_int.tmp_geo_reference

Query is running:   0%|          |

Downloading:   0%|          |

,geo_id,name,capital,_data_source,_load_time
0,NH,New Hampshire,Concord,noaa,2025-02-04 16:02:38.600918+00:00
1,ID,Idaho,Boise,noaa,2025-02-04 16:02:38.600918+00:00
2,IN,Indiana,Indianapolis,noaa,2025-02-04 16:02:38.600918+00:00
3,MD,Maryland,Annapolis,noaa,2025-02-04 16:02:38.600918+00:00
4,RI,Rhode Island,Providence,noaa,2025-02-04 16:02:38.600918+00:00
...,...,...,...,...,...
61,MP,Northern Mariana Islands,Saipan,usepa,2025-02-06 21:30:29.918903+00:00
62,USTERR,None,None,usepa,2025-02-06 21:30:29.918903+00:00
63,PW,Palau,Ngerulmud,usepa,2025-02-06 21:30:29.918903+00:00
64,MH,Marshall Islands,Majuro,usepa,2025-02-06 21:30:29.918903+00:00


Check uniqueness of Primary Key field (`geo_id`)

In [ ]:
%%bigquery
select geo_id, count(*) as count
from us_climate_int.tmp_geo_reference
group by geo_id
having count(*) > 1

Query is running:   0%|          |

Downloading: |          |

,geo_id,count


Create final table

In [ ]:
%%bigquery
create or replace table us_climate_int.Geo_Reference as
  select geo_id, name, capital, _data_source, _load_time
  from us_climate_int.tmp_geo_reference

Query is running:   0%|          |

""


In [ ]:
%%bigquery
create or replace table us_climate_int.Geo_References as
  select *
  from us_climate_int.Geo_Reference

Query is running:   0%|          |

""


Clean up

In [ ]:
%%bigquery
drop table if exists us_climate_int.tmp_geo_reference;
drop table if exists us_climate_int.Geo_Reference;

Query is running:   0%|          |

""


## Create `Facility_Ghg_Emission` table

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE us_climate_int.tmp_facility_ghg AS
SELECT *
FROM us_climate_stg.facility_ghg_emissions

Query is running:   0%|          |

""


Delete all records which have a null facility_id

In [ ]:
%%bigquery
DELETE FROM us_climate_int.tmp_facility_ghg WHERE facility_id IS NULL

Query is running:   0%|          |

""


### Link the `Facility_Ghg_Emissions` table to the `Organization` Table

Add the organization column

In [ ]:
%%bigquery
ALTER TABLE us_climate_int.tmp_facility_ghg
ADD COLUMN organization STRING;

Query is running:   0%|          |

""


Fill in the organization column with the corresponding name

In [ ]:
%%bigquery
UPDATE us_climate_int.tmp_facility_ghg AS f
SET organization = m.standardized_name
FROM us_climate_int.tmp_ghg_facilities_llm_org_checkpoint AS c
JOIN us_climate_int.tmp_ghg_organization_name_mapping AS m
    ON TRIM(c.organization_name) = TRIM(m.original_name)
WHERE f.facility_id = CAST(c.facility_id AS INT64);

Query is running:   0%|          |

""


Change the city names to lower cases

In [ ]:
%%bigquery
UPDATE us_climate_int.tmp_facility_ghg
SET city = INITCAP(LOWER(city))
WHERE city IS NOT NULL;

Query is running:   0%|          |

""


In [ ]:
%%bigquery
CREATE OR REPLACE TABLE us_climate_int.tmp_facility_ghg_reordered AS
SELECT
    facility_id,
    facility_name,
    organization,
    city,
    state,
    naics_code,
    year,
    industry_sector1,
    industry_sector2,
    industry_sector3,
    max_rated_heat_input_capacity,
    carbon_dioxide_emissions,
    methane_emissions,
    nitrous_oxide_emissions,
    biogenic_co2_emissions,
    _data_source,
    _load_time
FROM us_climate_int.tmp_facility_ghg;

Query is running:   0%|          |

""


In [ ]:
%%bigquery
DROP TABLE us_climate_int.tmp_facility_ghg;

Query is running:   0%|          |

""


In [ ]:
%%bigquery
ALTER TABLE us_climate_int.tmp_facility_ghg_reordered
RENAME TO tmp_facility_ghg;

Query is running:   0%|          |

""


### Create primary key and foreign key

Check uniqueness for the `facility_id`

In [ ]:
%%bigquery
SELECT facility_id, year, COUNT(*) AS count
FROM us_climate_int.tmp_facility_ghg
GROUP BY facility_id, facility_name, organization, city, state, naics_code, year, industry_sector1, industry_sector2, industry_sector3, max_rated_heat_input_capacity, carbon_dioxide_emissions, methane_emissions, nitrous_oxide_emissions,biogenic_co2_emissions
HAVING COUNT(*) > 1
ORDER BY count DESC;

Query is running:   0%|          |

Downloading:   0%|          |

,facility_id,year,count
0,1007989,2011,26
1,1007989,2012,26
2,1007794,2022,24
3,1007794,2020,24
4,1007794,2023,24
...,...,...,...
5444,1001890,2015,2
5445,1001561,2014,2
5446,1001561,2015,2
5447,1008027,2023,2


Rank the Duplicates

In [ ]:
%%bigquery
WITH ranked_data AS (
    SELECT *,
        ROW_NUMBER() OVER (
            PARTITION BY facility_id, year
            ORDER BY _load_time DESC
        ) AS rank
    FROM us_climate_int.tmp_facility_ghg
)
SELECT * FROM ranked_data
WHERE rank = 1;

Query is running:   0%|          |

Downloading:   0%|          |

,facility_id,facility_name,organization,city,state,naics_code,year,industry_sector1,industry_sector2,industry_sector3,max_rated_heat_input_capacity,carbon_dioxide_emissions,methane_emissions,nitrous_oxide_emissions,biogenic_co2_emissions,_data_source,_load_time,rank
0,1000017,BASF CORPORATION,BASF,Palmyra,MO,325320,2011,Chemicals,None,None,142.00,59569.7,175.50,303.960,0.0,usepa,2025-02-04 16:02:53.582757+00:00,1
1,1000019,WESTERN SUGAR COMPANY,Western Sugar Cooperative,Scottsbluff,NE,311313,2016,Minerals,None,None,28.00,4906.1,2.25,2.682,0.0,usepa,2025-02-04 16:02:53.582757+00:00,1
2,1000019,WESTERN SUGAR COMPANY,Western Sugar Cooperative,Scottsbluff,NE,311313,2018,Minerals,None,None,28.00,4906.1,2.25,2.682,0.0,usepa,2025-02-04 16:02:53.582757+00:00,1
3,1000039,GE's Grid Solutions,GE Grid Solutions,Charleroi,PA,335313,2021,Import and Export of Equipment Containing Fluo...,Other,None,0.80,156.9,0.00,0.000,0.0,usepa,2025-02-04 16:02:53.582757+00:00,1
4,1000351,"MW/MB, LLC",None,Clarksville,TN,327212,2012,Minerals,None,None,41.12,20502.8,9.75,11.920,0.0,usepa,2025-02-04 16:02:53.582757+00:00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78211,1003631,US ARMY FORT BLISS,Army,El Paso,TX,928110,2018,Other,None,None,0.15,1066.0,0.50,0.596,0.0,usepa,2025-02-04 16:02:53.582757+00:00,1
78212,1004219,U S MARINE CORPS CHERRY POINT AIR STATION,United States Marine Corps,Cherry Point,NC,928110,2023,Other,None,None,96.00,3493.9,3.50,8.344,0.0,usepa,2025-02-04 16:02:53.582757+00:00,1
78213,1004604,U.S. Army IMCOM and Fort Leonard Wood,United States Army,Fort Leonard Wood,MO,928110,2021,Waste,None,None,42.00,35823.0,17.00,20.264,0.0,usepa,2025-02-04 16:02:53.582757+00:00,1
78214,1005367,US ARMY - FORT BELVOIR,Department of the Army,Fort Belvoir,VA,928110,2015,Other,None,None,32.20,28805.1,15.00,21.158,0.0,usepa,2025-02-04 16:02:53.582757+00:00,1


Remove Duplicates and Keep Only Rank 1

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE us_climate_int.tmp_facility_ghg_deduplicated AS
WITH ranked_data AS (
    SELECT *,
        ROW_NUMBER() OVER (
            PARTITION BY facility_id, year
            ORDER BY facility_id DESC
        ) AS rank
    FROM us_climate_int.tmp_facility_ghg
)
SELECT * EXCEPT(rank)
FROM ranked_data
WHERE rank = 1;

Query is running:   0%|          |

""


Replace the Old Table

In [ ]:
%%bigquery
DROP TABLE us_climate_int.tmp_facility_ghg;

Query is running:   0%|          |

""


In [ ]:
%%bigquery
ALTER TABLE us_climate_int.tmp_facility_ghg_deduplicated
RENAME TO tmp_facility_ghg;

Query is running:   0%|          |

""


Verify That Duplicates Are Removed

In [ ]:
%%bigquery
SELECT facility_id, year, COUNT(*) AS count
FROM us_climate_int.tmp_facility_ghg
GROUP BY facility_id, year
HAVING COUNT(*) > 1;

Query is running:   0%|          |

Downloading: |          |

,facility_id,year,count


Check foreign key constraints for `organization`

In [ ]:
%%bigquery
SELECT COUNT(*) AS orphan_records
FROM us_climate_int.tmp_facility_ghg AS ghg
WHERE ghg.organization NOT IN (
    SELECT DISTINCT organization_name FROM us_climate_int.Organization
);

Query is running:   0%|          |

Downloading:   0%|          |

,orphan_records
0,0


Create the final `Facility_Ghg_Emissions` table

In [ ]:
%%bigquery
create or replace table us_climate_int.Facility_Ghg_Emissions as
  select *
  from us_climate_int.tmp_facility_ghg

Query is running:   0%|          |

""


In [ ]:
%%bigquery
create or replace table us_climate_int.Facility_GHG_Emission as
  select *
  from us_climate_int.Facility_Ghg_Emissions

Query is running:   0%|          |

""


Clean up

In [ ]:
%%bigquery
drop table if exists us_climate_int.tmp_facility_ghg;
drop table if exists us_climate_int.Facility_Ghg_Emissions

Query is running:   0%|          |

""


## Create `Carbon_Capture_Facility` table
### Create `Carbon_Capture_Categories` table (criteria 8)

In [ ]:
%%bigquery
select * from us_climate_stg.carbon_capture_facilities

Query is running:   0%|          |

Downloading:   0%|          |

,id,facility,organization,city,state,category,status,industry,_data_source,_load_time
0,54,Net Zero Hydrogen Energy Complex,Praduch,Edmonton,AB,"Capture, Storage, Transport",In Development,Hydrogen Production,bbi-international,2025-02-04 16:21:17.573644+00:00
1,52,Alberta Hydrogen Production and Liquidation Fa...,Air Products,Edmonton,AB,"Capture, Storage, Transport",In Development,Hydrogen Production,bbi-international,2025-02-04 16:21:17.573644+00:00
2,74,Capture Power Complex,Pleridae Energy,Calgary,AB,"Capture, Storage",In Development,Power Generation,bbi-international,2025-02-04 16:21:17.573644+00:00
3,55,Edmonton Hydrogen Capture facility,CapturePoint Solutions,Edmonton,AB,"Capture, Storage",In Development,Hydrogen Production,bbi-international,2025-02-04 16:21:17.573644+00:00
4,41,LSB El Dorado Facility,LSB El Dorado Facility,El Dorado,AR,"Capture, Storage",In Development,Ethanol Production,bbi-international,2025-02-04 16:21:17.573644+00:00
...,...,...,...,...,...,...,...,...,...,...
130,70,Hackberry Carbon Sequestration Project,Stakeholder Gas Services LLC,Hackberry,LA,"Capture, Storage, Transport",Proposed,Natural Gas Production,bbi-international,2025-02-04 16:21:17.573644+00:00
131,30,Ag LLC (Atkinson),Ag LLC,Atkinson,NE,"Capture, Storage, Transport",Proposed,Ethanol Production,bbi-international,2025-02-04 16:21:17.573644+00:00
132,48,Newell County Facility,Newell County Facility,Newell,SD,"Capture, Storage",Proposed,Ethanol Production,bbi-international,2025-02-04 16:21:17.573644+00:00
133,114,Rio Grande Liquefaction Hub Site,NextDecade,Rio Grande,TX,"Capture, Storage",Proposed,Natural Gas Production,bbi-international,2025-02-04 16:21:17.573644+00:00


In [ ]:
%%bigquery
select facility, count(*) as count
from us_climate_stg.carbon_capture_facilities
group by facility
having count(*) > 1

Query is running:   0%|          |

Downloading:   0%|          |

,facility,count
0,Coming,2
1,Summit Carbon,2
2,Summit Carbon Solutions,4
3,Testing Facility,3
4,Huron Carbon Capture & Storage Project,2
5,Open,6
6,Ash Grove Cement Plant,2


In [ ]:
%%bigquery
select * from us_climate_stg.carbon_capture_facilities where facility = 'Summit Carbon Solutions'

Query is running:   0%|          |

Downloading:   0%|          |

,id,facility,organization,city,state,category,status,industry,_data_source,_load_time
0,135,Summit Carbon Solutions,Summit Carbon Solutions,Goldfield,IA,"Capture, Storage",In Development,Ethanol Production,bbi-international,2025-02-04 16:21:17.573644+00:00
1,100,Summit Carbon Solutions,Summit Carbon Solutions,Merrill,IA,"Capture, Storage",In Development,Ethanol Production,bbi-international,2025-02-04 16:21:17.573644+00:00
2,23,Summit Carbon Solutions,Summit Carbon Solutions,Summit,SD,"Capture, Storage, Transport",In Development,Ethanol Production,bbi-international,2025-02-04 16:21:17.573644+00:00
3,113,Summit Carbon Solutions,Summit Carbon Solutions,Onida,SD,"Capture, Storage",In Development,Ethanol Production,bbi-international,2025-02-04 16:21:17.573644+00:00


Rows with the exact same data appear but have id different.

In [ ]:
%%bigquery
select facility, organization, city, state, category, status, industry, count(*) as count
from us_climate_stg.carbon_capture_facilities
group by facility, organization, city, state, category, status, industry
having count(*) > 1

Query is running:   0%|          |

Downloading:   0%|          |

,facility,organization,city,state,category,status,industry,count
0,Summit Carbon,Summit Carbon,Sioux City,IA,"Capture, Storage, Transport",In Development,Ethanol Production,2
1,Huron Carbon Capture & Storage Project,Summit Carbon Solutions,Huron,SD,"Capture, Storage, Transport",In Development,Ethanol Production,2
2,Open,Open,Open,IA,"Capture, Storage",Open,Ethanol Production,4


In [ ]:
%%bigquery
select * from us_climate_stg.carbon_capture_facilities where facility = 'Summit Carbon'

Query is running:   0%|          |

Downloading:   0%|          |

,id,facility,organization,city,state,category,status,industry,_data_source,_load_time
0,36,Summit Carbon,Summit Carbon,Sioux City,IA,"Capture, Storage, Transport",In Development,Ethanol Production,bbi-international,2025-02-04 16:21:17.573644+00:00
1,37,Summit Carbon,Summit Carbon,Sioux City,IA,"Capture, Storage, Transport",In Development,Ethanol Production,bbi-international,2025-02-04 16:21:17.573644+00:00


Create tmp table

In [ ]:
%%bigquery
create or replace table us_climate_int.tmp_ccf as
  select id,
    facility,
    organization,
    city,
    state,
    category as categories,
    status,
    industry,
    _data_source,
    _load_time
  from us_climate_stg.carbon_capture_facilities

Query is running:   0%|          |

""


Rank rows that have all but id the same

In [ ]:
%%bigquery
with ranked_rows as (
  select *,
         rank() over (partition by facility, organization, city, state, categories, status, industry order by id) as rank
  from us_climate_int.tmp_ccf
)
select * from ranked_rows
where rank > 1;

Query is running:   0%|          |

Downloading:   0%|          |

,id,facility,organization,city,state,categories,status,industry,_data_source,_load_time,rank
0,90,Open,Open,Open,IA,"Capture, Storage",Open,Ethanol Production,bbi-international,2025-02-04 16:21:17.573644+00:00,2
1,91,Open,Open,Open,IA,"Capture, Storage",Open,Ethanol Production,bbi-international,2025-02-04 16:21:17.573644+00:00,3
2,92,Open,Open,Open,IA,"Capture, Storage",Open,Ethanol Production,bbi-international,2025-02-04 16:21:17.573644+00:00,4
3,37,Summit Carbon,Summit Carbon,Sioux City,IA,"Capture, Storage, Transport",In Development,Ethanol Production,bbi-international,2025-02-04 16:21:17.573644+00:00,2
4,18,Huron Carbon Capture & Storage Project,Summit Carbon Solutions,Huron,SD,"Capture, Storage, Transport",In Development,Ethanol Production,bbi-international,2025-02-04 16:21:17.573644+00:00,2


In [ ]:
%%bigquery
create or replace table us_climate_int.tmp_ccf_cleaned as
  with ranked_rows as (
    select *,
           rank() over (partition by facility, organization, city, state, categories, status, industry order by id) as rank
   from us_climate_int.tmp_ccf
  )
  select * from ranked_rows
  where rank = 1;

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select facility, organization, city, state, categories, status, industry, count(*) as count
from us_climate_int.tmp_ccf_cleaned
group by facility, organization, city, state, categories, status, industry
having count(*) > 1

Query is running:   0%|          |

Downloading: |          |

,facility,organization,city,state,categories,status,industry,count


### Link the Carbon_Capture_Facilities table to the organization table

In [ ]:
%%bigquery
UPDATE us_climate_int.tmp_ccf_cleaned AS f
SET organization = m.standardized_name
FROM us_climate_int.tmp_ccf_facilities_llm_org_checkpoint AS c
JOIN us_climate_int.tmp_ccf_organization_name_mapping AS m
    ON TRIM(c.organization_name) = TRIM(m.original_name)
WHERE CAST(f.id AS STRING) = c.ccf_id;

Query is running:   0%|          |

""


### Create primary key and foreign key

Check `id` is unique

In [ ]:
%%bigquery
select id, count(*) as count
from us_climate_int.tmp_ccf_cleaned
group by id
having count(*) > 1

Query is running:   0%|          |

Downloading: |          |

,id,count


Check foreign key constraint (`organization`)

In [ ]:
%%bigquery
SELECT COUNT(*) AS orphan_records
FROM us_climate_int.tmp_ccf_cleaned AS ccf
WHERE ccf.organization NOT IN (
    SELECT DISTINCT organization_name FROM us_climate_int.Organization
);

Query is running:   0%|          |

Downloading:   0%|          |

,orphan_records
0,0


Create final `Carbon_Capture_Facility` table

In [ ]:
%%bigquery
create or replace table us_climate_int.Carbon_Capture_Facility as
  select *
  from us_climate_int.tmp_ccf_cleaned

Query is running:   0%|          |

""


Clean up

In [ ]:
%%bigquery
drop table if exists us_climate_int.tmp_ccf;
drop table if exists us_climate_int.tmp_ccf_cleaned;

Query is running:   0%|          |

""


### Create Carbon_Capture_Categories table

In [ ]:
%%bigquery
create or replace table us_climate_int.tmp_categories as
  select id,
    facility,
    organization,
    city,
    state,
    categories,
    status,
    industry,
    _data_source,
    _load_time
  from us_climate_int.Carbon_Capture_Facility

Query is running:   0%|          |

""


Convert `categories` into an array (criteria 8)

In [ ]:
%%bigquery
select id, facility, categories, split(categories, ',') as categories_array
from us_climate_int.tmp_categories

Query is running:   0%|          |

Downloading:   0%|          |

,id,facility,categories,categories_array
0,8,Navajo Generating Station,Capture,[Capture]
1,3,EW Brown Generating Plant,Capture,[Capture]
2,2,Enid Nitrogen Facility,Capture,[Capture]
3,6,Fluar,Capture,[Capture]
4,50,Nutrien Redwater Fertilizer,"Capture, Storage","[Capture, Storage]"
...,...,...,...,...
125,12,Huron Carbon Capture & Storage Project,"Capture, Storage, Transport","[Capture, Storage, Transport]"
126,126,Coastal Bend Carbon Management Partnership,"Capture, Storage, Transport","[Capture, Storage, Transport]"
127,71,Campo Viejo Gas Processing Plant,"Capture, Storage, Transport","[Capture, Storage, Transport]"
128,14,Grand Prairie Carbon Capture & Storage Project,"Capture, Storage, Transport","[Capture, Storage, Transport]"


In [ ]:
%%bigquery
with categories as (select id as facility_id, split(categories, ',') as categories_array, _data_source, _load_time
                    from us_climate_int.tmp_categories)
select facility_id, category, _data_source, _load_time
from categories, unnest(categories_array) as category

Query is running:   0%|          |

Downloading:   0%|          |

,facility_id,category,_data_source,_load_time
0,8,Capture,bbi-international,2025-02-04 16:21:17.573644+00:00
1,3,Capture,bbi-international,2025-02-04 16:21:17.573644+00:00
2,2,Capture,bbi-international,2025-02-04 16:21:17.573644+00:00
3,6,Capture,bbi-international,2025-02-04 16:21:17.573644+00:00
4,50,Capture,bbi-international,2025-02-04 16:21:17.573644+00:00
...,...,...,...,...
283,14,Storage,bbi-international,2025-02-04 16:21:17.573644+00:00
284,14,Transport,bbi-international,2025-02-04 16:21:17.573644+00:00
285,56,Capture,bbi-international,2025-02-04 16:21:17.573644+00:00
286,56,Storage,bbi-international,2025-02-04 16:21:17.573644+00:00


Create final table

In [ ]:
%%bigquery
create or replace table us_climate_int.Carbon_Capture_Categories as
  with categories as (select id as facility_id, split(categories, ',') as categories_array, _data_source, _load_time
                      from us_climate_int.tmp_categories)
  select facility_id, category, _data_source, _load_time
  from categories, unnest(categories_array) as category

Query is running:   0%|          |

""


Check Primary Key Field (`facility_id` and `category`)

In [ ]:
%%bigquery
select facility_id, category, count(*) as count
from us_climate_int.Carbon_Capture_Categories
group by facility_id, category
having count(*) > 1

Query is running:   0%|          |

Downloading: |          |

,facility_id,category,count


Drop `categories` from `Carbon_Capture_Facility` table

In [ ]:
%%bigquery
alter table us_climate_int.Carbon_Capture_Facility
    drop column categories

Query is running:   0%|          |

""


Clean up

In [ ]:
%%bigquery
drop table if exists us_climate_int.tmp_categories

Query is running:   0%|          |

""


## `Climate_Risk_Projection` table (criteria 9)
`climate_risk_projections` currently only has county code as an identifier, need to map county code to state using `county_codes` table to be able to create a relationship with other tables.

In [ ]:
%%bigquery
select r.county_code, c.county_code, c.state, c.county
from us_climate_stg.climate_risk_projections r
left join us_climate_stg.county_codes c
on r.county_code = c.county_code

Query is running:   0%|          |

Downloading:   0%|          |

,county_code,county_code_1,state,county
0,06075,06075,California,San Francisco
1,06041,06041,California,Marin
2,06013,06013,California,Contra Costa
3,06067,06067,California,Sacramento
4,34017,34017,New Jersey,Hudson
...,...,...,...,...
3103,45049,45049,South Carolina,Hampton
3104,45089,45089,South Carolina,Williamsburg
3105,28109,28109,Mississippi,Pearl River
3106,13185,13185,Georgia,Lowndes


In [ ]:
%%bigquery
select r.county_code,
    c.state,
    c.county,
    r.temp_change,
    r.precipitation_change,
    r.extreme_precipitation,
    r.extreme_cold,
    r.extreme_heat,
    r.dry_change,
    r.impervious_surface,
    r.housing_density,
    r.population_estimate,
    r.low_lying_houses,
    r.low_lying_roads,
    r.hazard,
    r.exposure,
    r.vulnerability,
    r.risk_percentage,
    r._data_source,
    r._load_time
from us_climate_stg.climate_risk_projections r
left join us_climate_stg.county_codes c
on r.county_code = c.county_code

Query is running:   0%|          |

Downloading:   0%|          |

,county_code,state,county,temp_change,precipitation_change,extreme_precipitation,extreme_cold,extreme_heat,dry_change,impervious_surface,housing_density,population_estimate,low_lying_houses,low_lying_roads,hazard,exposure,vulnerability,risk_percentage,_data_source,_load_time
0,06075,California,San Francisco,1.329085,-1.414940,40.0,54.6,13.8,-0.421788,52.593117,28960.133660,6.693903e+05,0.001750,0.000867,0.062776,0.439533,0.210377,0.580476,nasa,2025-02-19 20:11:48.411172+00:00
1,06041,California,Marin,1.392268,-0.109442,68.3,92.5,17.0,-0.291676,5.775580,1417.648027,2.084583e+05,0.050250,0.033935,0.225861,0.086940,0.131711,0.258632,nasa,2025-02-19 20:11:48.411172+00:00
2,06013,California,Contra Costa,1.296070,0.198513,100.5,101.4,27.0,-0.244542,13.063562,3568.211362,1.395204e+06,0.023355,0.121703,0.287772,0.205985,0.326997,1.938328,nasa,2025-02-19 20:11:48.411172+00:00
3,06067,California,Sacramento,1.417000,-2.202548,12.4,59.3,28.4,-0.349909,12.746265,3341.231409,1.920577e+06,0.071107,0.226935,0.101481,0.288938,0.436043,1.278548,nasa,2025-02-19 20:11:48.411172+00:00
4,34017,New Jersey,Hudson,1.935840,0.317492,140.1,57.9,28.8,-0.003940,51.136787,39661.580870,1.217110e+06,0.045399,0.004756,0.271299,0.487572,0.270736,3.581233,nasa,2025-02-19 20:11:48.411172+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3103,45049,South Carolina,Hampton,1.602453,0.350678,121.3,39.3,57.2,-0.124159,1.108010,115.986525,1.441000e+04,0.000066,0.000003,0.264996,0.008094,0.310509,0.066598,nasa,2025-02-19 20:11:48.411172+00:00
3104,45089,South Carolina,Williamsburg,1.521935,0.202708,120.3,45.3,59.2,-0.130511,0.999027,92.980016,2.222900e+04,0.000033,0.000003,0.271766,0.007525,0.313210,0.064052,nasa,2025-02-19 20:11:48.411172+00:00
3105,28109,Mississippi,Pearl River,1.568313,0.505275,90.0,40.8,62.8,-0.138883,1.372386,141.630786,6.098433e+04,0.000000,0.000003,0.274922,0.011398,0.318603,0.099839,nasa,2025-02-19 20:11:48.411172+00:00
3106,13185,Georgia,Lowndes,1.469570,0.381588,110.0,40.7,69.9,-0.107506,3.272546,434.799898,1.405117e+05,0.000000,0.000003,0.287705,0.027597,0.309415,0.245668,nasa,2025-02-19 20:11:48.411172+00:00


In [ ]:
%%bigquery
create or replace table us_climate_int.tmp_crp as
  select r.county_code,
    c.state,
    c.county,
    r.temp_change,
    r.precipitation_change,
    r.extreme_precipitation,
    r.extreme_cold,
    r.extreme_heat,
    r.dry_change,
    r.impervious_surface,
    r.housing_density,
    r.population_estimate,
    r.low_lying_houses,
    r.low_lying_roads,
    r.hazard,
    r.exposure,
    r.vulnerability,
    r.risk_percentage,
    r._data_source,
    r._load_time
  from us_climate_stg.climate_risk_projections r
  left join us_climate_stg.county_codes c
  on r.county_code = c.county_code

Query is running:   0%|          |

""


Change state to our unique identifier

In [ ]:
%%bigquery
select c.county_code, c.state, s.geo_id, s.name
from us_climate_int.tmp_crp c
left join us_climate_int.Geo_Reference s
on c.state = s.name

Query is running:   0%|          |

Downloading:   0%|          |

,county_code,state,geo_id,name
0,01061,Alabama,AL,Alabama
1,01097,Alabama,AL,Alabama
2,01025,Alabama,AL,Alabama
3,01099,Alabama,AL,Alabama
4,01003,Alabama,AL,Alabama
...,...,...,...,...
3103,51800,Virginia,VA,Virginia
3104,51119,Virginia,VA,Virginia
3105,51093,Virginia,VA,Virginia
3106,53011,Washington,WA,Washington


In [ ]:
%%bigquery
create or replace table us_climate_int.tmp_crp as
  select c.county_code,
    s.geo_id as state,
    c.county,
    c.temp_change,
    c.precipitation_change,
    c.extreme_precipitation,
    c.extreme_cold,
    c.extreme_heat,
    c.dry_change,
    c.impervious_surface,
    c.housing_density,
    c.population_estimate,
    c.low_lying_houses,
    c.low_lying_roads,
    c.hazard,
    c.exposure,
    c.vulnerability,
    c.risk_percentage,
    c._data_source,
    c._load_time
  from us_climate_int.tmp_crp c
  left join us_climate_int.Geo_Reference s
  on c.state = s.name

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select * from us_climate_int.tmp_crp

Query is running:   0%|          |

Downloading:   0%|          |

,county_code,state,county,temp_change,precipitation_change,extreme_precipitation,extreme_cold,extreme_heat,dry_change,impervious_surface,housing_density,population_estimate,low_lying_houses,low_lying_roads,hazard,exposure,vulnerability,risk_percentage,_data_source,_load_time
0,01097,AL,Mobile,1.619940,0.321084,103.2,41.5,64.2,-0.139217,4.571746,774.888434,541690.000000,0.005408,0.086353,0.285679,0.091240,0.336897,0.878133,nasa,2025-02-19 20:11:48.411172+00:00
1,01129,AL,Washington,1.464152,0.511680,96.8,52.8,68.9,-0.135721,0.567115,38.696663,9855.000000,0.000013,0.006735,0.313705,0.007079,0.310900,0.069040,nasa,2025-02-19 20:11:48.411172+00:00
2,01003,AL,Baldwin,1.405428,0.406291,99.9,45.1,61.7,-0.127586,2.775429,343.216029,281908.000000,0.007427,0.197105,0.264107,0.120874,0.304934,0.973462,nasa,2025-02-19 20:11:48.411172+00:00
3,06113,CA,Yolo,1.789032,0.120068,80.5,53.0,43.5,-0.238032,3.148472,504.773712,417035.000000,0.000413,0.065995,0.261292,0.065520,0.350765,0.600502,nasa,2025-02-19 20:11:48.411172+00:00
4,06087,CA,Santa Cruz,1.641188,0.292869,79.1,96.8,39.8,-0.280036,8.156975,1720.132713,413699.000000,0.000832,0.002263,0.347970,0.073752,0.354796,0.910524,nasa,2025-02-19 20:11:48.411172+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3103,56011,WY,Crook,1.847895,0.143156,75.7,62.0,29.8,-0.226338,0.120620,8.831154,5167.666667,0.000000,0.000000,0.250813,0.000729,0.251373,0.004600,nasa,2025-02-19 20:11:48.411172+00:00
3104,56025,WY,Natrona,2.225142,-1.125237,22.6,60.4,33.0,-0.323917,0.193022,130.907619,49635.666670,0.000000,0.000000,0.235827,0.002842,0.226527,0.015184,nasa,2025-02-19 20:11:48.411172+00:00
3105,56019,WY,Johnson,1.874015,-0.981961,22.2,64.6,24.6,-0.309399,0.089212,17.673105,5013.333333,0.000000,0.000000,0.186590,0.000523,0.263442,0.002570,nasa,2025-02-19 20:11:48.411172+00:00
3106,56037,WY,Sweetwater,2.200782,-3.240102,0.0,75.2,32.7,-0.384410,0.111114,152.129757,17459.000000,0.000000,0.000000,0.209281,0.001309,0.243264,0.006660,nasa,2025-02-19 20:11:48.411172+00:00


Check Primary Key field (`county_code`)

In [ ]:
%%bigquery
select county_code, count(*) as count
from us_climate_int.tmp_crp
group by county_code
having count(*) > 1

Query is running:   0%|          |

Downloading: |          |

,county_code,count


Create final table

In [ ]:
%%bigquery
create or replace table us_climate_int.Climate_Risk_Projections as
  select county_code,
    state,
    temp_change,
    precipitation_change,
    extreme_precipitation,
    extreme_cold,
    extreme_heat,
    dry_change,
    impervious_surface,
    housing_density,
    population_estimate,
    low_lying_houses,
    low_lying_roads,
    hazard,
    exposure,
    vulnerability,
    risk_percentage,
    _data_source,
    _load_time
  from us_climate_int.tmp_crp

Query is running:   0%|          |

""


In [ ]:
%%bigquery
create or replace table us_climate_int.Climate_Risk_Projection as
  select *
  from us_climate_int.Climate_Risk_Projections

Query is running:   0%|          |

""


Clean up

In [ ]:
%%bigquery
drop table if exists us_climate_int.tmp_crp;
drop table if exists us_climate_int.Climate_Risk_Projections;

Query is running:   0%|          |

""


## `State_Average_Temperature` table

Need to change `state` to new identifier

In [ ]:
%%bigquery
select * from us_climate_stg.state_average_temperature

Query is running:   0%|          |

Downloading:   0%|          |

,month,year,state,average_temp,monthly_mean_from_1901_to_2000,_data_source,_load_time
0,1,1967,Alabama,45.6,45.8,kaggle,2025-02-04 16:02:30.320061+00:00
1,1,2013,Alabama,50.4,45.8,kaggle,2025-02-04 16:02:30.320061+00:00
2,1,2021,Alabama,46.2,45.8,kaggle,2025-02-04 16:02:30.320061+00:00
3,1,1981,Alabama,42.2,45.8,kaggle,2025-02-04 16:02:30.320061+00:00
4,1,1965,Alabama,47.4,45.8,kaggle,2025-02-04 16:02:30.320061+00:00
...,...,...,...,...,...,...,...
41851,8,1979,Wyoming,64.2,64.4,kaggle,2025-02-04 16:02:30.320061+00:00
41852,8,2007,Wyoming,68.6,64.4,kaggle,2025-02-04 16:02:30.320061+00:00
41853,8,2011,Wyoming,66.9,64.4,kaggle,2025-02-04 16:02:30.320061+00:00
41854,8,2016,Wyoming,65.2,64.4,kaggle,2025-02-04 16:02:30.320061+00:00


In [ ]:
%%bigquery
select t.month,
  t.year,
  s.geo_id,
  t.average_temp,
  t.monthly_mean_from_1901_to_2000,
  t._data_source,
  t._load_time
from us_climate_stg.state_average_temperature t
left join us_climate_int.Geo_Reference s
on t.state = s.name

Query is running:   0%|          |

Downloading:   0%|          |

,month,year,geo_id,average_temp,monthly_mean_from_1901_to_2000,_data_source,_load_time
0,1,1967,AL,45.6,45.8,kaggle,2025-02-04 16:02:30.320061+00:00
1,1,2013,AL,50.4,45.8,kaggle,2025-02-04 16:02:30.320061+00:00
2,1,2021,AL,46.2,45.8,kaggle,2025-02-04 16:02:30.320061+00:00
3,1,1981,AL,42.2,45.8,kaggle,2025-02-04 16:02:30.320061+00:00
4,1,1965,AL,47.4,45.8,kaggle,2025-02-04 16:02:30.320061+00:00
...,...,...,...,...,...,...,...
41851,8,1979,WY,64.2,64.4,kaggle,2025-02-04 16:02:30.320061+00:00
41852,8,2007,WY,68.6,64.4,kaggle,2025-02-04 16:02:30.320061+00:00
41853,8,2011,WY,66.9,64.4,kaggle,2025-02-04 16:02:30.320061+00:00
41854,8,2016,WY,65.2,64.4,kaggle,2025-02-04 16:02:30.320061+00:00


In [ ]:
%%bigquery
create or replace table us_climate_int.tmp_sat as
  select t.month,
    t.year,
    s.geo_id as state,
    t.average_temp,
    t.monthly_mean_from_1901_to_2000,
    t._data_source,
    t._load_time
  from us_climate_stg.state_average_temperature t
  left join us_climate_int.Geo_Reference s
  on t.state = s.name

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select * from us_climate_int.tmp_sat

Query is running:   0%|          |

Downloading:   0%|          |

,month,year,state,average_temp,monthly_mean_from_1901_to_2000,_data_source,_load_time
0,1,2006,AL,47.2,45.8,kaggle,2025-02-04 16:02:30.320061+00:00
1,1,2018,AL,43.6,45.8,kaggle,2025-02-04 16:02:30.320061+00:00
2,1,1984,AL,41.0,45.8,kaggle,2025-02-04 16:02:30.320061+00:00
3,1,2001,AL,40.2,45.8,kaggle,2025-02-04 16:02:30.320061+00:00
4,1,1974,AL,50.5,45.8,kaggle,2025-02-04 16:02:30.320061+00:00
...,...,...,...,...,...,...,...
41851,8,1962,WY,62.4,64.4,kaggle,2025-02-04 16:02:30.320061+00:00
41852,8,1967,WY,64.2,64.4,kaggle,2025-02-04 16:02:30.320061+00:00
41853,8,2011,WY,66.9,64.4,kaggle,2025-02-04 16:02:30.320061+00:00
41854,8,2007,WY,68.6,64.4,kaggle,2025-02-04 16:02:30.320061+00:00


Check Primary Key Fields (`month`, `year`, `state`)

In [ ]:
%%bigquery
select month, year, state, count(*) as count
from us_climate_int.tmp_sat
group by month, year, state
having count(*) > 1

Query is running:   0%|          |

Downloading: |          |

,month,year,state,count


Create final table

In [ ]:
%%bigquery
create or replace table us_climate_int.State_Average_Temperature as
  select
    month,
    year,
    state,
    average_temp,
    monthly_mean_from_1901_to_2000,
    _data_source,
    _load_time
  from us_climate_int.tmp_sat

Query is running:   0%|          |

""


Clean up

In [ ]:
%%bigquery
drop table if exists us_climate_int.tmp_sat

Query is running:   0%|          |

""


## `State_Climate_Policy` table (criteria 9)
### `Policies` table (criteria 10)

Look to see how many unique policies there are

In [ ]:
%%bigquery
select distinct policy from us_climate_stg.state_climate_policies

Query is running:   0%|          |

Downloading:   0%|          |

,policy
0,Agriculture Financial Incentives
1,Agriculture Technical Assistance Programs
2,All-Electric Buildings
3,Appliance Standards
4,Bicycle Friendly States
...,...
81,All-Electric Buildings Policies
82,All-Electric Buildings Standards
83,Commission on Environmental Justice and Sustai...
84,Net Metering


In [ ]:
%%bigquery
select distinct policy_area from us_climate_stg.state_climate_policies

Query is running:   0%|          |

Downloading:   0%|          |

,policy_area
0,Natural and Working Lands
1,Buildings and Efficiency
2,Transportation
3,"Industry, Materials, and Waste Management"
4,Cross-Sector
5,Electricity
6,Climate Governance and Equity
7,Climate
8,"Transmission, Distribution, and Energy Storage"


In [ ]:
%%bigquery
select distinct category from us_climate_stg.state_climate_policies

Query is running:   0%|          |

Downloading:   0%|          |

,category
0,Agriculture
1,Building Electrification
2,Building Standards
3,Public and Active Transportation
4,Industrial Decarbonization
5,Carbon Valuation
6,Energy Plans and Targets
7,Climate Governance
8,Coal Retirement
9,Building Codes


In [ ]:
%%bigquery
select policy, policy_area, category from us_climate_stg.state_climate_policies where policy = 'Agriculture Financial Incentives'

Query is running:   0%|          |

Downloading:   0%|          |

,policy,policy_area,category
0,Agriculture Financial Incentives,Natural and Working Lands,Agriculture
1,Agriculture Financial Incentives,Natural and Working Lands,Agriculture
2,Agriculture Financial Incentives,Natural and Working Lands,Agriculture
3,Agriculture Financial Incentives,Natural and Working Lands,Agriculture
4,Agriculture Financial Incentives,Natural and Working Lands,Agriculture
5,Agriculture Financial Incentives,Natural and Working Lands,Agriculture
6,Agriculture Financial Incentives,Natural and Working Lands,Agriculture
7,Agriculture Financial Incentives,Natural and Working Lands,Agriculture
8,Agriculture Financial Incentives,Natural and Working Lands,Agriculture
9,Agriculture Financial Incentives,Natural and Working Lands,Agriculture


Create `Policies` table, since `policy`, `policy_area`, and `category` seem to be grouped uniquely (criteria 10)

In [ ]:
%%bigquery
select distinct policy, policy_area, category from us_climate_stg.state_climate_policies

Query is running:   0%|          |

Downloading:   0%|          |

,policy,policy_area,category
0,Agriculture Financial Incentives,Natural and Working Lands,Agriculture
1,Agriculture Technical Assistance Programs,Natural and Working Lands,Agriculture
2,All-Electric Buildings,Buildings and Efficiency,Building Electrification
3,Appliance Standards,Buildings and Efficiency,Building Standards
4,Bicycle Friendly States,Transportation,Public and Active Transportation
...,...,...,...
104,Just Transition Funds,Climate,Just Transition
105,Just Transition Offices and Staff,Climate,Just Transition
106,Just Transition Plans,Climate,Just Transition
107,Net Metering,Electricity,Clean Energy Generation


In [ ]:
%%bigquery
create or replace table us_climate_int.tmp_policy as
  select distinct policy, policy_area, category
  from us_climate_stg.state_climate_policies

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select policy, count(*) as count
from us_climate_int.tmp_policy
group by policy
having count(*) > 1

Query is running:   0%|          |

Downloading:   0%|          |

,policy,count
0,Energy Efficiency Resource Standards (EERS),2
1,Climate Action Plans,2
2,Greenhouse Gas Emissions Reduction Targets,2
3,State Greenhouse Gas Emissions Inventory,2
4,Climate Bureaucracy,2
5,Climate Advisory Bodies,2
6,Environmental Justice Community Definitions,3
7,Environmental Justice Community Investment Req...,3
8,Cumulative Impact Assessment,3
9,Environmental Justice Mapping Tools,3


In [ ]:
%%bigquery
select policy, policy_area, category
from us_climate_int.tmp_policy
where policy = 'Energy Efficiency Resource Standards (EERS)'

Query is running:   0%|          |

Downloading:   0%|          |

,policy,policy_area,category
0,Energy Efficiency Resource Standards (EERS),Buildings and Efficiency,Building Efficiency
1,Energy Efficiency Resource Standards (EERS),Buildings and Efficiency,Building Standards


In [ ]:
%%bigquery
select policy, policy_area, category
from us_climate_int.tmp_policy
where policy = 'Environmental Justice Bureaucracy'

Query is running:   0%|          |

Downloading:   0%|          |

,policy,policy_area,category
0,Environmental Justice Bureaucracy,Climate,Environmental Justice
1,Environmental Justice Bureaucracy,Climate,Environmental Justice and Equity
2,Environmental Justice Bureaucracy,Climate Governance and Equity,Environmental Justice and Equity


In [ ]:
%%bigquery
select policy, policy_area, category
from us_climate_int.tmp_policy
where policy = 'Cumulative Impact Assessment'

Query is running:   0%|          |

Downloading:   0%|          |

,policy,policy_area,category
0,Cumulative Impact Assessment,Climate,Environmental Justice
1,Cumulative Impact Assessment,Climate,Environmental Justice and Equity
2,Cumulative Impact Assessment,Climate Governance and Equity,Environmental Justice and Equity


In [ ]:
%%bigquery
select policy, policy_area, category
from us_climate_int.tmp_policy
where policy = 'Interconnection Standards'

Query is running:   0%|          |

Downloading:   0%|          |

,policy,policy_area,category
0,Interconnection Standards,Electricity,"Transmission, Distribution, and Energy Storage"
1,Interconnection Standards,"Transmission, Distribution, and Energy Storage",Electricity


In [ ]:
%%bigquery
select policy, policy_area, category
from us_climate_int.tmp_policy
where policy = 'Just Transition Advisory Bodies'

Query is running:   0%|          |

Downloading:   0%|          |

,policy,policy_area,category
0,Just Transition Advisory Bodies,Climate,Just Transition
1,Just Transition Advisory Bodies,Climate Governance and Equity,Just Transition


Create new tmp table with same data before llm to compare

In [ ]:
%%bigquery
create or replace table us_climate_int.tmp_main_policy as
  select distinct policy, policy_area, category
  from us_climate_stg.state_climate_policies

Query is running:   0%|          |

""


Use LLM to remove redundancy in `policy_area` and `category` and standardize values

In [ ]:
import json
import pandas as pd
import pandas_gbq
from google.cloud import bigquery
import vertexai
from vertexai.generative_models import GenerativeModel

# Define the prompt for the LLM
prompt = """For each policy entry, standardize the `policy_area` and `category` values while keeping `policy` unchanged.

Standardization rules:
1. If `policy_area` is "Climate" or "Climate Governance and Equity", standardize it to "Climate Governance and Equity".
2. If `category` is "Environmental Justice" or "Environmental Justice and Equity", standardize it to "Environmental Justice and Equity".
3. If `policy_area` is "Buildings and Efficiency" and the `category` is "Building Efficiency" standardize `category` to "Building Efficiency and Standards".
4. If `policy_area` is "Buildings and Efficiency" and the `category` is "Building Standards" standardize `category` to "Building Efficiency and Standards".
4. If `policy_area` is "Electricity" and `category` is "Transmission, Distribution, and Energy Storage" keep values unchanged.
5. If `policy_area` is "Transmission, Distribution, and Energy Storage" and `category` is "Electricity" swap there values so that:
   - `policy_area` becomes "Electricity"
   - `category` becomes "Transmission, Distribution, and Energy Storage"
6. If no standardization is needed, keep the values unchanged.

Format the answer as a list of dictionaries with the schema:
{"policy": "<policy>", "policy_area": "<standardized policy_area>", "category": "<standardized category>"}

Example:
[{"policy": "Just Transition Advisory Bodies	", "policy_area": "Climate Governance and Equity	", "category": "Just Transition"},
 {"policy": "Interconnection Standards", "policy_area": "Electricity", "category": "Transmission, Distribution, and Energy Storage"}]
"""

bq_client = bigquery.Client()

input_sql = """
SELECT policy, policy_area, category
FROM us_climate_int.tmp_policy
"""

input_list = []
rows = bq_client.query(input_sql).result()

for row in rows:
    input_list.append({
        "policy": row["policy"],
        "policy_area": row["policy_area"],
        "category": row["category"]
    })

input_str = json.dumps(input_list)

vertexai.init(project=project_id, location=region)
model = GenerativeModel(model_name)
resp = model.generate_content([input_str, prompt])
resp_text = resp.text.replace("```json", "").replace("```", "").replace("\n", "")
print("results_raw:", resp_text)
replacements = json.loads(resp_text)
print("replacements:", replacements)

df = pd.DataFrame(replacements)

table_id = "us_climate_int.tmp_policy"
pandas_gbq.to_gbq(df, table_id, project_id=project_id, if_exists="replace")

results_raw: [  {"policy": "Stretch Building Energy Codes", "policy_area": "Buildings and Efficiency", "category": "Building Codes"},  {"policy": "Residential Energy Codes", "policy_area": "Buildings and Efficiency", "category": "Building Codes"},  {"policy": "Commercial Energy Codes", "policy_area": "Buildings and Efficiency", "category": "Building Codes"},  {"policy": "Property Assessed Clean Energy (PACE)", "policy_area": "Buildings and Efficiency", "category": "Building Efficiency and Standards"},  {"policy": "Weatherization Funding", "policy_area": "Buildings and Efficiency", "category": "Building Efficiency and Standards"},  {"policy": "Energy Efficiency Resource Standards (EERS)", "policy_area": "Buildings and Efficiency", "category": "Building Efficiency and Standards"},  {"policy": "All-Electric Buildings", "policy_area": "Buildings and Efficiency", "category": "Building Electrification"},  {"policy": "All-Electric Buildings Policies", "policy_area": "Buildings and Efficiency"

100%|██████████| 1/1 [00:00<00:00, 7973.96it/s]


Check if only things prompted to LLM were changed

In [ ]:
%%bigquery
select
  m.policy,
  m.policy_area,
  m.category,
  p.policy,
  p.policy_area,
  p.category
from us_climate_int.tmp_main_policy m
join us_climate_int.tmp_policy p
on m.policy = p.policy
where m.policy_area != p.policy_area

Query is running:   0%|          |

Downloading:   0%|          |

,policy,policy_area,category,policy_1,policy_area_1,category_1
0,Climate Advisory Team (CAT),Climate,Climate Governance,Climate Advisory Team (CAT),Climate Governance and Equity,Climate Governance
1,State Greenhouse Gas Emissions Inventory,Climate,Climate Governance,State Greenhouse Gas Emissions Inventory,Climate Governance and Equity,Climate Governance
2,State Greenhouse Gas Emissions Inventory,Climate,Climate Governance,State Greenhouse Gas Emissions Inventory,Climate Governance and Equity,Climate Governance
3,Climate Action Plans,Climate,Climate Governance,Climate Action Plans,Climate Governance and Equity,Climate Governance
4,Climate Action Plans,Climate,Climate Governance,Climate Action Plans,Climate Governance and Equity,Climate Governance
5,Greenhouse Gas Emissions Reduction Targets,Climate,Climate Governance,Greenhouse Gas Emissions Reduction Targets,Climate Governance and Equity,Climate Governance
6,Greenhouse Gas Emissions Reduction Targets,Climate,Climate Governance,Greenhouse Gas Emissions Reduction Targets,Climate Governance and Equity,Climate Governance
7,Climate Advisory Bodies,Climate,Climate Governance,Climate Advisory Bodies,Climate Governance and Equity,Climate Governance
8,Climate Advisory Bodies,Climate,Climate Governance,Climate Advisory Bodies,Climate Governance and Equity,Climate Governance
9,NC Climate Change Interagency Council,Climate,Climate Governance,NC Climate Change Interagency Council,Climate Governance and Equity,Climate Governance


In [ ]:
%%bigquery
select
  m.policy,
  m.policy_area,
  m.category,
  p.policy,
  p.policy_area,
  p.category
from us_climate_int.tmp_main_policy m
join us_climate_int.tmp_policy p
on m.policy = p.policy
where m.category != p.category

Query is running:   0%|          |

Downloading:   0%|          |

,policy,policy_area,category,policy_1,policy_area_1,category_1
0,Weatherization Funding,Buildings and Efficiency,Building Efficiency,Weatherization Funding,Buildings and Efficiency,Building Efficiency and Standards
1,Energy Efficiency Resource Standards (EERS),Buildings and Efficiency,Building Efficiency,Energy Efficiency Resource Standards (EERS),Buildings and Efficiency,Building Efficiency and Standards
2,Energy Efficiency Resource Standards (EERS),Buildings and Efficiency,Building Efficiency,Energy Efficiency Resource Standards (EERS),Buildings and Efficiency,Building Efficiency and Standards
3,Property Assessed Clean Energy (PACE),Buildings and Efficiency,Building Efficiency,Property Assessed Clean Energy (PACE),Buildings and Efficiency,Building Efficiency and Standards
4,Building Performance Standards,Buildings and Efficiency,Building Standards,Building Performance Standards,Buildings and Efficiency,Building Efficiency and Standards
5,Appliance Standards,Buildings and Efficiency,Building Standards,Appliance Standards,Buildings and Efficiency,Building Efficiency and Standards
6,Clean Heat Standards,Buildings and Efficiency,Building Standards,Clean Heat Standards,Buildings and Efficiency,Building Efficiency and Standards
7,Energy Efficiency Resource Standards (EERS),Buildings and Efficiency,Building Standards,Energy Efficiency Resource Standards (EERS),Buildings and Efficiency,Building Efficiency and Standards
8,Energy Efficiency Resource Standards (EERS),Buildings and Efficiency,Building Standards,Energy Efficiency Resource Standards (EERS),Buildings and Efficiency,Building Efficiency and Standards
9,Environmental Justice Community Definitions,Climate,Environmental Justice,Environmental Justice Community Definitions,Climate Governance and Equity,Environmental Justice and Equity


In [ ]:
%%bigquery
select
  m.policy,
  m.policy_area,
  m.category,
  p.policy,
  p.policy_area,
  p.category
from us_climate_int.tmp_main_policy m
join us_climate_int.tmp_policy p
on m.policy = p.policy
where m.policy != p.policy

Query is running:   0%|          |

Downloading: |          |

,policy,policy_area,category,policy_1,policy_area_1,category_1


In [ ]:
%%bigquery
select policy, count(*) as count
from us_climate_int.tmp_policy
group by policy
having count(*) > 1

Query is running:   0%|          |

Downloading:   0%|          |

,policy,count
0,Energy Efficiency Resource Standards (EERS),2
1,Climate Action Plans,2
2,Greenhouse Gas Emissions Reduction Targets,2
3,State Greenhouse Gas Emissions Inventory,2
4,Climate Bureaucracy,2
5,Climate Advisory Bodies,2
6,Environmental Justice Community Definitions,3
7,Environmental Justice Community Investment Req...,3
8,Cumulative Impact Assessment,3
9,Environmental Justice Mapping Tools,3


In [ ]:
%%bigquery
select policy, policy_area, category
from us_climate_int.tmp_policy
where policy = 'Energy Efficiency Resource Standards (EERS)'

Query is running:   0%|          |

Downloading:   0%|          |

,policy,policy_area,category
0,Energy Efficiency Resource Standards (EERS),Buildings and Efficiency,Building Efficiency and Standards
1,Energy Efficiency Resource Standards (EERS),Buildings and Efficiency,Building Efficiency and Standards


Need to remove duplicate values now

In [ ]:
%%bigquery
create or replace table us_climate_int.tmp_policy as
  select distinct policy, policy_area, category
  from us_climate_int.tmp_policy

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select * from us_climate_int.tmp_policy

Query is running:   0%|          |

Downloading:   0%|          |

,policy,policy_area,category
0,Residential Energy Codes,Buildings and Efficiency,Building Codes
1,Stretch Building Energy Codes,Buildings and Efficiency,Building Codes
2,Commercial Energy Codes,Buildings and Efficiency,Building Codes
3,Clean Heat Standards,Buildings and Efficiency,Building Efficiency and Standards
4,Energy Efficiency Resource Standards (EERS),Buildings and Efficiency,Building Efficiency and Standards
...,...,...,...
81,Medium- and Heavy-Duty Electric Vehicle (EV) R...,Transportation,Medium- and Heavy-Duty Vehicles
82,Bicycle Friendly States,Transportation,Public and Active Transportation
83,Electric Vehicle (EV) and EV Charging Infrastr...,Transportation,Transportation Plans and Targets
84,Transportation Greenhouse Gas Emissions Reduct...,Transportation,Transportation Plans and Targets


Check Primary Key field (`policy`)

In [ ]:
%%bigquery
select policy, count(*) as count
from us_climate_int.tmp_policy
group by policy
having count(*) > 1

Query is running:   0%|          |

Downloading: |          |

,policy,count


Create final `Policies` table

In [ ]:
%%bigquery
create or replace table us_climate_int.Policies as
  select distinct
    p.policy,
    p.policy_area,
    p.category,
    s._data_source,
    s._load_time
  from us_climate_int.tmp_policy p
  left join us_climate_stg.state_climate_policies s
  on p.policy = s.policy

Query is running:   0%|          |

""


Change `state` in `State_Climate_Policies` table

In [ ]:
%%bigquery
select * from us_climate_stg.state_climate_policies

Query is running:   0%|          |

Downloading:   0%|          |

,state,policy,policy_area,category,status,year_enacted,_data_source,_load_time
0,Maine,Agriculture Financial Incentives,Natural and Working Lands,Agriculture,Enacted,2021,climatexchange,2025-02-04 16:15:14.794411+00:00
1,New York,Agriculture Financial Incentives,Natural and Working Lands,Agriculture,Enacted,2021,climatexchange,2025-02-04 16:15:14.794411+00:00
2,Illinois,Agriculture Financial Incentives,Natural and Working Lands,Agriculture,Enacted,<NA>,climatexchange,2025-02-04 16:15:14.794411+00:00
3,Iowa,Agriculture Financial Incentives,Natural and Working Lands,Agriculture,Enacted,2020,climatexchange,2025-02-04 16:15:14.794411+00:00
4,Maryland,Agriculture Financial Incentives,Natural and Working Lands,Agriculture,Enacted,2017,climatexchange,2025-02-04 16:15:14.794411+00:00
...,...,...,...,...,...,...,...,...
3092,Minnesota,Weatherization Funding,Buildings and Efficiency,Building Efficiency,Partially Enacted,<NA>,climatexchange,2025-02-04 16:15:14.794411+00:00
3093,Montana,Weatherization Funding,Buildings and Efficiency,Building Efficiency,Partially Enacted,<NA>,climatexchange,2025-02-04 16:15:14.794411+00:00
3094,Vermont,Commercial Energy Codes,Buildings and Efficiency,Building Codes,Unlisted,2024,climatexchange,2025-02-04 16:15:14.794411+00:00
3095,Vermont,Energy Efficiency Resource Standards (EERS),Buildings and Efficiency,Building Efficiency,Unlisted,2023,climatexchange,2025-02-04 16:15:14.794411+00:00


In [ ]:
%%bigquery
create or replace table us_climate_int.tmp_scp as
  select
    s.geo_id as state,
    p.policy,
    p.status,
    p.year_enacted,
    p._data_source,
    p._load_time
  from us_climate_stg.state_climate_policies p
  left join us_climate_int.Geo_Reference s
  on p.state = s.name

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select * from us_climate_int.tmp_scp

Query is running:   0%|          |

Downloading:   0%|          |

,state,policy,status,year_enacted,_data_source,_load_time
0,VA,All-Electric Buildings Policies,Not Enacted,<NA>,climatexchange,2025-02-04 16:15:14.794411+00:00
1,MD,Commission on Environmental Justice and Sustai...,Not Enacted,<NA>,climatexchange,2025-02-04 16:15:14.794411+00:00
2,TN,All-Electric Buildings Policies,Not Enacted,<NA>,climatexchange,2025-02-04 16:15:14.794411+00:00
3,AK,All-Electric Buildings Standards,Not Enacted,<NA>,climatexchange,2025-02-04 16:15:14.794411+00:00
4,ME,Equity Subcommittee of the Maine Climate Council,Enacted,<NA>,climatexchange,2025-02-04 16:15:14.794411+00:00
...,...,...,...,...,...,...
3092,PA,State Greenhouse Gas Emissions Inventory,Enacted,2024,climatexchange,2025-02-04 16:15:14.794411+00:00
3093,IL,Stretch Building Energy Codes,Enacted,2024,climatexchange,2025-02-04 16:15:14.794411+00:00
3094,VT,Stretch Building Energy Codes,Enacted,2024,climatexchange,2025-02-04 16:15:14.794411+00:00
3095,CO,Transportation Greenhouse Gas Emissions Reduct...,Enacted,2024,climatexchange,2025-02-04 16:15:14.794411+00:00


`policy` in both tables match

In [ ]:
%%bigquery
select s.policy, p.policy
from us_climate_int.tmp_scp s
join us_climate_int.Policy p
on s.policy = p.policy
where s.policy != p.policy

Query is running:   0%|          |

Downloading: |          |

,policy,policy_1


Check Primary Key field (`state` and `policy`)

In [ ]:
%%bigquery
select state, policy, count(*) as count
from us_climate_int.tmp_scp
group by state, policy
having count(*) > 1

Query is running:   0%|          |

Downloading: |          |

,state,policy,count


Create final table

In [ ]:
%%bigquery
create or replace table us_climate_int.State_Climate_Policies as
  select
    state,
    policy,
    status,
    year_enacted,
    _data_source,
    _load_time
  from us_climate_int.tmp_scp

Query is running:   0%|          |

""


In [ ]:
%%bigquery
create or replace table us_climate_int.State_Climate_Policy as
  select *
  from us_climate_int.State_Climate_Policies

Query is running:   0%|          |

""


Clean up

In [ ]:
%%bigquery
drop table if exists us_climate_int.tmp_main_policy;
drop table if exists us_climate_int.tmp_policy;
drop table if exists us_climate_int.tmp_scp;
drop table if exists us_climate_int.State_Climate_Policies;

Query is running:   0%|          |

""


## `State_Disaster` table


In [ ]:
%%bigquery
select * from us_climate_stg.state_disasters

Query is running:   0%|          |

Downloading:   0%|          |

,state,year,drought_count,drought_cost_min,drought_cost_max,flooding_count,flooding_cost_min,flooding_cost_max,freeze_count,freeze_cost_min,...,wildfire_cost_min,wildfire_cost_max,winter_storm_count,winter_storm_cost_min,winter_storm_cost_max,all_disasters_count,all_disasters_cost_min,all_disasters_cost_max,_data_source,_load_time
0,NY,1980,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,noaa,2025-02-04 16:02:38.600918+00:00
1,CT,1980,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,noaa,2025-02-04 16:02:38.600918+00:00
2,VT,1980,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,noaa,2025-02-04 16:02:38.600918+00:00
3,WV,1980,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,noaa,2025-02-04 16:02:38.600918+00:00
4,PA,1980,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,noaa,2025-02-04 16:02:38.600918+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2245,IL,2024,0,0,0,1,5,100,0,0,...,0,0,1,100,250,12,2000,5000,noaa,2025-02-04 16:02:38.600918+00:00
2246,GA,2024,1,5,100,0,0,0,0,0,...,0,0,1,5,100,13,10000,20000,noaa,2025-02-04 16:02:38.600918+00:00
2247,NC,2024,1,250,500,0,0,0,0,0,...,0,0,0,0,0,9,20000,50000,noaa,2025-02-04 16:02:38.600918+00:00
2248,TX,2024,1,1000,2000,0,0,0,0,0,...,0,0,1,250,500,20,20000,50000,noaa,2025-02-04 16:02:38.600918+00:00


In [ ]:
%%bigquery
create or replace table us_climate_int.tmp_state_disasters as
  select state,
    year,
    drought_count,
    drought_cost_min,
    drought_cost_max,
    flooding_count,
    flooding_cost_min,
    flooding_cost_max,
    freeze_count,
    freeze_cost_min,
    freeze_cost_max,
    severe_storm_count,
    severe_storm_cost_min,
    severe_storm_cost_max,
    tropical_cyclone_count,
    tropical_cyclone_cost_min,
    tropical_cyclone_cost_max,
    wildfire_count,
    wildfire_cost_min,
    wildfire_cost_max,
    winter_storm_count,
    winter_storm_cost_min,
    winter_storm_cost_max,
    all_disasters_count,
    all_disasters_cost_min,
    all_disasters_cost_max,
    _data_source,
    _load_time
  from us_climate_stg.state_disasters

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select * from us_climate_int.tmp_state_disasters

Query is running:   0%|          |

Downloading:   0%|          |

,state,year,drought_count,drought_cost_min,drought_cost_max,flooding_count,flooding_cost_min,flooding_cost_max,freeze_count,freeze_cost_min,...,wildfire_cost_min,wildfire_cost_max,winter_storm_count,winter_storm_cost_min,winter_storm_cost_max,all_disasters_count,all_disasters_cost_min,all_disasters_cost_max,_data_source,_load_time
0,FL,1980,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,noaa,2025-02-04 16:02:38.600918+00:00
1,CT,1980,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,noaa,2025-02-04 16:02:38.600918+00:00
2,DE,1980,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,noaa,2025-02-04 16:02:38.600918+00:00
3,ME,1980,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,noaa,2025-02-04 16:02:38.600918+00:00
4,UT,1980,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,noaa,2025-02-04 16:02:38.600918+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2245,TN,2024,1,100,250,0,0,0,0,0,...,0,0,1,250,500,8,2000,5000,noaa,2025-02-04 16:02:38.600918+00:00
2246,GA,2024,1,5,100,0,0,0,0,0,...,0,0,1,5,100,13,10000,20000,noaa,2025-02-04 16:02:38.600918+00:00
2247,NC,2024,1,250,500,0,0,0,0,0,...,0,0,0,0,0,9,20000,50000,noaa,2025-02-04 16:02:38.600918+00:00
2248,TX,2024,1,1000,2000,0,0,0,0,0,...,0,0,1,250,500,20,20000,50000,noaa,2025-02-04 16:02:38.600918+00:00


Check Primary Key fields (`state`, `year`)

In [ ]:
%%bigquery
select state, year, count(*) as count
from us_climate_int.tmp_state_disasters
group by state, year
having count(*) > 1

Query is running:   0%|          |

Downloading: |          |

,state,year,count


Create final table

In [ ]:
%%bigquery
create or replace table us_climate_int.State_Disasters as
  select state,
    year,
    drought_count,
    drought_cost_min,
    drought_cost_max,
    flooding_count,
    flooding_cost_min,
    flooding_cost_max,
    freeze_count,
    freeze_cost_min,
    freeze_cost_max,
    severe_storm_count,
    severe_storm_cost_min,
    severe_storm_cost_max,
    tropical_cyclone_count,
    tropical_cyclone_cost_min,
    tropical_cyclone_cost_max,
    wildfire_count,
    wildfire_cost_min,
    wildfire_cost_max,
    winter_storm_count,
    winter_storm_cost_min,
    winter_storm_cost_max,
    all_disasters_count,
    all_disasters_cost_min,
    all_disasters_cost_max,
    _data_source,
    _load_time
  from us_climate_int.tmp_state_disasters

Query is running:   0%|          |

""


In [ ]:
%%bigquery
create or replace table us_climate_int.State_Disaster as
  select *
  from us_climate_int.State_Disasters

Query is running:   0%|          |

""


Clean up

In [ ]:
%%bigquery
drop table if exists us_climate_int.tmp_state_disasters;
drop table if exists us_climate_int.State_Disasters;

Query is running:   0%|          |

""


## `State_Electricity_Generation_Fuel_Share` table

Need to change `state` to new identifier

In [ ]:
%%bigquery
select * from us_climate_stg.state_electricity_generation_fuel_shares

Query is running:   0%|          |

Downloading:   0%|          |

,state,nuclear,coal,natural_gas,petroleum,hydro,geothermal,solar_power,wind,biomass_and_others,_data_source,_load_time
0,Colorado,0.0,41.6,25.5,0.0,2.8,0.0,3.1,26.5,0.4,nei,2025-02-04 16:26:33.592472+00:00
1,Oklahoma,0.0,14.0,40.8,0.0,3.3,0.0,0.1,41.4,0.4,nei,2025-02-04 16:26:33.592472+00:00
2,Mississippi,17.1,8.0,72.1,0.0,0.0,0.0,0.6,0.0,2.1,nei,2025-02-04 16:26:33.592472+00:00
3,District of Columbia,0.0,0.0,62.5,0.0,0.0,0.0,11.9,0.0,25.5,nei,2025-02-04 16:26:33.592472+00:00
4,Illinois,53.3,23.9,11.6,0.0,0.1,0.0,0.3,10.3,0.5,nei,2025-02-04 16:26:33.592472+00:00
5,Arizona,29.1,13.2,44.4,0.0,5.4,0.0,6.2,1.5,0.2,nei,2025-02-04 16:26:33.592472+00:00
6,Texas,8.3,18.4,48.6,0.0,0.2,0.0,2.9,20.7,0.9,nei,2025-02-04 16:26:33.592472+00:00
7,Alabama,32.2,18.8,37.6,0.0,8.8,0.0,0.3,0.0,2.2,nei,2025-02-04 16:26:33.592472+00:00
8,Washington,7.8,2.9,14.4,0.0,64.6,0.0,0.0,8.7,1.6,nei,2025-02-04 16:26:33.592472+00:00
9,South Dakota,0.0,9.2,8.7,0.1,29.7,0.0,0.0,52.3,0.1,nei,2025-02-04 16:26:33.592472+00:00


Need to fix "New York2" and "Michigan1"

In [ ]:
%%bigquery
create or replace table us_climate_int.tmp_elec as
  select
    case
      when state = 'New York2' then 'New York'
      when state = 'Michigan1' then 'Michigan'
      else state
    end as state,
    nuclear,
    coal,
    natural_gas,
    petroleum,
    hydro,
    geothermal,
    solar_power,
    wind,
    biomass_and_others,
    _data_source,
    _load_time
from us_climate_stg.state_electricity_generation_fuel_shares

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select * from us_climate_int.tmp_elec

Query is running:   0%|          |

Downloading:   0%|          |

,state,nuclear,coal,natural_gas,petroleum,hydro,geothermal,solar_power,wind,biomass_and_others,_data_source,_load_time
0,District of Columbia,0.0,0.0,62.5,0.0,0.0,0.0,11.9,0.0,25.5,nei,2025-02-04 16:26:33.592472+00:00
1,Arizona,29.1,13.2,44.4,0.0,5.4,0.0,6.2,1.5,0.2,nei,2025-02-04 16:26:33.592472+00:00
2,Mississippi,17.1,8.0,72.1,0.0,0.0,0.0,0.6,0.0,2.1,nei,2025-02-04 16:26:33.592472+00:00
3,Texas,8.3,18.4,48.6,0.0,0.2,0.0,2.9,20.7,0.9,nei,2025-02-04 16:26:33.592472+00:00
4,Washington,7.8,2.9,14.4,0.0,64.6,0.0,0.0,8.7,1.6,nei,2025-02-04 16:26:33.592472+00:00
5,Colorado,0.0,41.6,25.5,0.0,2.8,0.0,3.1,26.5,0.4,nei,2025-02-04 16:26:33.592472+00:00
6,Illinois,53.3,23.9,11.6,0.0,0.1,0.0,0.3,10.3,0.5,nei,2025-02-04 16:26:33.592472+00:00
7,Oklahoma,0.0,14.0,40.8,0.0,3.3,0.0,0.1,41.4,0.4,nei,2025-02-04 16:26:33.592472+00:00
8,Alabama,32.2,18.8,37.6,0.0,8.8,0.0,0.3,0.0,2.2,nei,2025-02-04 16:26:33.592472+00:00
9,Nebraska,17.8,49.2,4.1,0.1,3.3,0.0,0.1,25.2,0.2,nei,2025-02-04 16:26:33.592472+00:00


In [ ]:
%%bigquery
select e.state, s.geo_id
from us_climate_int.tmp_elec e
left join us_climate_int.Geo_Reference s
on e.state = s.name

Query is running:   0%|          |

Downloading:   0%|          |

,state,geo_id
0,District of Columbia,DC
1,Arizona,AZ
2,Mississippi,MS
3,Texas,TX
4,Washington,WA
5,Colorado,CO
6,Illinois,IL
7,Oklahoma,OK
8,Alabama,AL
9,Nebraska,NE


In [ ]:
%%bigquery
create or replace table us_climate_int.tmp_elec as
  select
    s.geo_id as state,
    e.nuclear,
    e.coal,
    e.natural_gas,
    e.petroleum,
    e.hydro,
    e.geothermal,
    e.solar_power,
    e.wind,
    e.biomass_and_others,
    e._data_source,
    e._load_time
  from us_climate_int.tmp_elec e
  left join us_climate_int.Geo_Reference s
  on e.state = s.name

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select * from us_climate_int.tmp_elec

Query is running:   0%|          |

Downloading:   0%|          |

,state,nuclear,coal,natural_gas,petroleum,hydro,geothermal,solar_power,wind,biomass_and_others,_data_source,_load_time
0,AZ,29.1,13.2,44.4,0.0,5.4,0.0,6.2,1.5,0.2,nei,2025-02-04 16:26:33.592472+00:00
1,CO,0.0,41.6,25.5,0.0,2.8,0.0,3.1,26.5,0.4,nei,2025-02-04 16:26:33.592472+00:00
2,MS,17.1,8.0,72.1,0.0,0.0,0.0,0.6,0.0,2.1,nei,2025-02-04 16:26:33.592472+00:00
3,IL,53.3,23.9,11.6,0.0,0.1,0.0,0.3,10.3,0.5,nei,2025-02-04 16:26:33.592472+00:00
4,WA,7.8,2.9,14.4,0.0,64.6,0.0,0.0,8.7,1.6,nei,2025-02-04 16:26:33.592472+00:00
5,AL,32.2,18.8,37.6,0.0,8.8,0.0,0.3,0.0,2.2,nei,2025-02-04 16:26:33.592472+00:00
6,OK,0.0,14.0,40.8,0.0,3.3,0.0,0.1,41.4,0.4,nei,2025-02-04 16:26:33.592472+00:00
7,DC,0.0,0.0,62.5,0.0,0.0,0.0,11.9,0.0,25.5,nei,2025-02-04 16:26:33.592472+00:00
8,TX,8.3,18.4,48.6,0.0,0.2,0.0,2.9,20.7,0.9,nei,2025-02-04 16:26:33.592472+00:00
9,NJ,45.8,1.7,47.8,0.1,0.0,0.0,2.5,0.0,2.3,nei,2025-02-04 16:26:33.592472+00:00


Check Primary Key field (`state`)

In [ ]:
%%bigquery
select state, count(*) as count
from us_climate_int.tmp_elec
group by state
having count(*) > 1

Query is running:   0%|          |

Downloading: |          |

,state,count


Create final table

In [ ]:
%%bigquery
create or replace table us_climate_int.State_Electricity_Generation_Fuel_Shares as
  select
    state,
    nuclear,
    coal,
    natural_gas,
    petroleum,
    hydro,
    geothermal,
    solar_power,
    wind,
    biomass_and_others,
    _data_source,
    _load_time
  from us_climate_int.tmp_elec

Query is running:   0%|          |

""


In [ ]:
%%bigquery
create or replace table us_climate_int.State_Electricity_Generation_Fuel_Share as
  select *
  from us_climate_int.State_Electricity_Generation_Fuel_Shares

Query is running:   0%|          |

""


Clean up

In [ ]:
%%bigquery
drop table if exists us_climate_int.tmp_elec;
drop table if exists us_climate_int.State_Electricity_Generation_Fuel_Shares;

Query is running:   0%|          |

""


## `State_GHG_Emission` table

In [ ]:
%%bigquery
create or replace table us_climate_int.State_GHG_Emissions as
  select
    geo_ref,
    sector,
    subsector,
    econ_sector,
    econ_subsector,
    category,
    sub_category_1,
    sub_category_2,
    sub_category_3,
    fuel,
    ghg,
    Y1990, Y1991, Y1992, Y1993, Y1994, Y1995, Y1996, Y1997, Y1998, Y1999,
    Y2000, Y2001, Y2002, Y2003, Y2004, Y2005, Y2006, Y2007, Y2008, Y2009,
    Y2010, Y2011, Y2012, Y2013, Y2014, Y2015, Y2016, Y2017, Y2018, Y2019,
    Y2020, Y2021, Y2022,
    _data_source,
    _load_time
  from us_climate_stg.state_ghg_emissions

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select * from us_climate_int.State_GHG_Emissions

Query is running:   0%|          |

Downloading:   0%|          |

,geo_ref,sector,subsector,econ_sector,econ_subsector,category,sub_category_1,sub_category_2,sub_category_3,fuel,...,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020,Y2021,Y2022,_data_source,_load_time
0,OK,Energy,Fugitives,Industry,Abandoned Underground Coal Mines,Abandoned Coal Mines,Recovered &Used,None,None,None,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,usepa,2025-02-06 21:30:29.918903+00:00
1,MS,Energy,Fugitives,Industry,Abandoned Oil and Gas Wells,Abandoned Wells,Abandoned Wells,Gas,None,None,...,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,usepa,2025-02-06 21:30:29.918903+00:00
2,IN,Energy,Fugitives,Industry,Abandoned Oil and Gas Wells,Abandoned Wells,Abandoned Wells,Gas,None,None,...,0.033322,0.033322,0.033322,0.033322,0.033322,0.033322,0.033322,0.033322,usepa,2025-02-06 21:30:29.918903+00:00
3,SC,Energy,Fugitives,Industry,Abandoned Oil and Gas Wells,Abandoned Wells,Abandoned Wells,Gas,None,None,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,usepa,2025-02-06 21:30:29.918903+00:00
4,HI,Agriculture,Direct and indirect N2O emissions from agricul...,Agriculture,Nitrous Oxide from Agricultural Soil Management,Agricultural Soil Management,Grassland,Indirect,None,None,...,0.011644,0.012237,0.012379,0.012520,0.012523,0.012401,0.012953,0.012987,usepa,2025-02-06 21:30:29.918903+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27213,UT,"Land Use, Land-Use Change and Forestry",Wetlands,LULUCF Sector Net Total,None,Wetlands Remaining Wetlands,Coastal Wetlands Remaining Coastal Wetlands,Vegetated Remaining Vegetated,None,None,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,usepa,2025-02-06 21:30:29.918903+00:00
27214,KS,"Land Use, Land-Use Change and Forestry",Wetlands,LULUCF Sector Net Total,None,Wetlands Remaining Wetlands,Coastal Wetlands Remaining Coastal Wetlands,Vegetated Converted to Open,None,None,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,usepa,2025-02-06 21:30:29.918903+00:00
27215,IN,"Land Use, Land-Use Change and Forestry",Wetlands,LULUCF Sector Net Total,None,Wetlands Remaining Wetlands,Coastal Wetlands Remaining Coastal Wetlands,Vegetated Remaining Vegetated,None,None,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,usepa,2025-02-06 21:30:29.918903+00:00
27216,MA,Industrial Processes and Product Use,Metal Industry,Industry,Zinc Production,Zinc Production,None,None,None,None,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,usepa,2025-02-06 21:30:29.918903+00:00


Check what Primary Key would be

In [ ]:
%%bigquery
select geo_ref, sector, subsector, econ_sector, econ_subsector, count(*) as count
from us_climate_int.State_GHG_Emissions
group by geo_ref, sector, subsector, econ_sector, econ_subsector
having count(*) > 1

Query is running:   0%|          |

Downloading:   0%|          |

,geo_ref,sector,subsector,econ_sector,econ_subsector,count
0,OK,Energy,Fugitives,Industry,Abandoned Underground Coal Mines,2
1,MS,Energy,Fugitives,Industry,Abandoned Oil and Gas Wells,4
2,IN,Energy,Fugitives,Industry,Abandoned Oil and Gas Wells,4
3,SC,Energy,Fugitives,Industry,Abandoned Oil and Gas Wells,4
4,HI,Agriculture,Direct and indirect N2O emissions from agricul...,Agriculture,Nitrous Oxide from Agricultural Soil Management,4
...,...,...,...,...,...,...
2495,NC,Agriculture,"CO2 Emissions from Liming, Urea Application an...",Agriculture,Liming,2
2496,AZ,Agriculture,Fossil Fuel Combustion,Industry,Stationary Combustion,2
2497,NY,Energy,Fossil Fuel Combustion,Residential,Carbon Dioxide from Fossil Fuel Combustion,3
2498,AR,Agriculture,Fossil Fuel Combustion,Industry,Stationary Combustion,2


In [ ]:
%%bigquery
select geo_ref, sector, subsector, category, sub_category_1, sub_category_2, count(*) as count
from us_climate_int.State_GHG_Emissions
group by geo_ref, sector, subsector, category, sub_category_1, sub_category_2
having count(*) > 1

Query is running:   0%|          |

Downloading:   0%|          |

,geo_ref,sector,subsector,category,sub_category_1,sub_category_2,count
0,TX,"Land Use, Land-Use Change and Forestry",Cropland,Land Converted to Cropland,Grassland Converted to Cropland,None,5
1,NV,"Land Use, Land-Use Change and Forestry",Grassland,Land Converted to Grassland,Other Lands Converted to Grassland,None,5
2,CA,"Land Use, Land-Use Change and Forestry",Grassland,Land Converted to Grassland,Other Lands Converted to Grassland,None,5
3,MN,"Land Use, Land-Use Change and Forestry",Cropland,Land Converted to Cropland,Wetlands Converted to Cropland,None,2
4,IN,"Land Use, Land-Use Change and Forestry",Cropland,Land Converted to Cropland,Grassland Converted to Cropland,None,2
...,...,...,...,...,...,...,...
5762,ID,"Land Use, Land-Use Change and Forestry",Wetlands,Flooded Land Remaining Flooded Land,other constructed waterbodies,canals and ditches,2
5763,ME,"Land Use, Land-Use Change and Forestry",Wetlands,Flooded Land Remaining Flooded Land,other constructed waterbodies,freshwater pond,2
5764,NV,"Land Use, Land-Use Change and Forestry",Wetlands,Flooded Land Remaining Flooded Land,other constructed waterbodies,canals and ditches,2
5765,UM,"Land Use, Land-Use Change and Forestry",Wetlands,Flooded Land Remaining Flooded Land,other constructed waterbodies,freshwater pond,2


In [ ]:
%%bigquery
select geo_ref, sector, subsector, ghg, count(*) as count
from us_climate_int.State_GHG_Emissions
group by geo_ref, sector, subsector, ghg
having count(*) > 1

Query is running:   0%|          |

Downloading:   0%|          |

,geo_ref,sector,subsector,ghg,count
0,OK,Energy,Fugitives,Methane,19
1,MS,Energy,Fugitives,Carbon Dioxide,14
2,IN,Energy,Fugitives,Methane,19
3,SC,Energy,Fugitives,Methane,14
4,HI,Agriculture,Direct and indirect N2O emissions from agricul...,Nitrous Oxide,4
...,...,...,...,...,...
3121,UT,Industrial Processes and Product Use,Chemical Industry,HFC-23,2
3122,GA,"Land Use, Land-Use Change and Forestry",Forest Land,Nitrous Oxide,2
3123,PA,"Land Use, Land-Use Change and Forestry",Forest Land,Methane,2
3124,TN,Industrial Processes and Product Use,Fossil Fuel Combustion,Nitrous Oxide,2


In [ ]:
%%bigquery
select geo_ref, sector, subsector, ghg, econ_sector, econ_subsector, sub_category_1, sub_category_2, count(*) as count
from us_climate_int.State_GHG_Emissions
group by geo_ref, sector, subsector, ghg, econ_sector, econ_subsector, sub_category_1, sub_category_2
having count(*) > 1

Query is running:   0%|          |

Downloading:   0%|          |

,geo_ref,sector,subsector,ghg,econ_sector,econ_subsector,sub_category_1,sub_category_2,count
0,CT,Energy,Fossil Fuel Combustion,Carbon Dioxide,Commercial,Carbon Dioxide from Fossil Fuel Combustion,None,None,3
1,MN,Energy,Fossil Fuel Combustion,Methane,Commercial,Stationary Combustion,None,None,4
2,MO,Energy,Fossil Fuel Combustion,Carbon Dioxide,Commercial,Carbon Dioxide from Fossil Fuel Combustion,None,None,3
3,VT,Energy,Fossil Fuel Combustion,Methane,Commercial,Stationary Combustion,None,None,4
4,NJ,Energy,Fossil Fuel Combustion,Carbon Dioxide,Commercial,Carbon Dioxide from Fossil Fuel Combustion,None,None,3
...,...,...,...,...,...,...,...,...,...
2719,NC,"Land Use, Land-Use Change and Forestry",Grassland,Carbon Dioxide,LULUCF Sector Net Total,None,Cropland Converted to Grassland,None,2
2720,NC,Agriculture,Field Burning of Agricultural Residues,Methane,Agriculture,Field Burning of Agricultural Residues,Tubers and Roots,Other,2
2721,AZ,Agriculture,Field Burning of Agricultural Residues,Nitrous Oxide,Agriculture,Field Burning of Agricultural Residues,Tubers and Roots,Other,2
2722,WA,"Land Use, Land-Use Change and Forestry",Wetlands,Carbon Dioxide,LULUCF Sector Net Total,None,Coastal Wetlands Remaining Coastal Wetlands,Vegetated Remaining Vegetated,2


In [ ]:
%%bigquery
select geo_ref, ghg, count(*) as count
from us_climate_int.State_GHG_Emissions
group by geo_ref, ghg
having count(*) > 1

Query is running:   0%|          |

Downloading:   0%|          |

,geo_ref,ghg,count
0,OK,Methane,120
1,MS,Carbon Dioxide,175
2,IN,Methane,120
3,SC,Methane,115
4,HI,Nitrous Oxide,70
...,...,...,...
1181,NM,HFC-32,2
1182,MH,HFC-32,2
1183,OH,PFC-3-1-10 (Perfluorobutane),2
1184,NC,HFC-143a,2


In [ ]:
%%bigquery
select geo_ref, ghg, sector, subsector, econ_sector, econ_subsector, sub_category_1, fuel, COUNT(*) AS count
from us_climate_int.State_GHG_Emissions
group by geo_ref, ghg, sector, subsector, econ_sector, econ_subsector, sub_category_1, fuel
having count(*) > 1

Query is running:   0%|          |

Downloading:   0%|          |

,geo_ref,ghg,sector,subsector,econ_sector,econ_subsector,sub_category_1,fuel,count
0,MS,Carbon Dioxide,Energy,Fugitives,Industry,Abandoned Oil and Gas Wells,Abandoned Wells,None,2
1,IN,Methane,Energy,Fugitives,Industry,Abandoned Oil and Gas Wells,Abandoned Wells,None,2
2,SC,Methane,Energy,Fugitives,Industry,Abandoned Oil and Gas Wells,Abandoned Wells,None,2
3,HI,Nitrous Oxide,Agriculture,Direct and indirect N2O emissions from agricul...,Agriculture,Nitrous Oxide from Agricultural Soil Management,Grassland,None,2
4,IN,Nitrous Oxide,Agriculture,Direct and indirect N2O emissions from agricul...,Agriculture,Nitrous Oxide from Agricultural Soil Management,Grassland,None,2
...,...,...,...,...,...,...,...,...,...
2344,NC,Carbon Dioxide,"Land Use, Land-Use Change and Forestry",Grassland,LULUCF Sector Net Total,None,Cropland Converted to Grassland,None,2
2345,NV,PFC-14 (Perfluoromethane),Industrial Processes and Product Use,Other product manufacture and use,Industry,SF6 and PFCs from Other Product Use,Scientific Applications,None,2
2346,NC,Carbon Dioxide,Energy,Fugitives,Industry,Abandoned Oil and Gas Wells,Abandoned Wells,None,2
2347,NC,Methane,Agriculture,Field Burning of Agricultural Residues,Agriculture,Field Burning of Agricultural Residues,Tubers and Roots,None,2


Rank the Duplicates

In [ ]:
%%bigquery
WITH ranked_data AS (
    SELECT *,
        ROW_NUMBER() OVER (
            PARTITION BY geo_ref, ghg, sector, subsector, econ_sector, econ_subsector, sub_category_1, fuel
            ORDER BY geo_ref DESC
        ) AS rank
    FROM us_climate_int.State_GHG_Emissions
)
SELECT * FROM ranked_data
WHERE rank = 1;

Query is running:   0%|          |

Downloading:   0%|          |

,geo_ref,sector,subsector,econ_sector,econ_subsector,category,sub_category_1,sub_category_2,sub_category_3,fuel,...,Y2016,Y2017,Y2018,Y2019,Y2020,Y2021,Y2022,_data_source,_load_time,rank
0,AS,Industrial Processes and Product Use,Metal Industry,Industry,Lead Production,Lead Production,None,None,None,None,...,0.00000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,usepa,2025-02-06 21:30:29.918903+00:00,1
1,DC,Industrial Processes and Product Use,Metal Industry,Industry,Zinc Production,Zinc Production,None,None,None,None,...,0.00000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,usepa,2025-02-06 21:30:29.918903+00:00,1
2,GU,Energy,Fossil Fuel Combustion,U.S. Territories,Stationary Combustion,US Territories,None,None,None,Coal,...,0.00000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,usepa,2025-02-06 21:30:29.918903+00:00,1
3,HI,Waste,Biological Treatment of Solid Waste,Commercial,Anaerobic Digestion at Biogas Facilities,Anaerobic Digestion at Biogas Facilities,None,None,None,None,...,0.00000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,usepa,2025-02-06 21:30:29.918903+00:00,1
4,MT,Waste,Biological Treatment of Solid Waste,Commercial,Anaerobic Digestion at Biogas Facilities,Anaerobic Digestion at Biogas Facilities,None,None,None,None,...,0.00000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,usepa,2025-02-06 21:30:29.918903+00:00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19966,OR,"Land Use, Land-Use Change and Forestry",Wetlands,LULUCF Sector Net Total,None,Wetlands Remaining Wetlands,Peatlands,Drained + Drainage Ditches,None,None,...,0.00000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,usepa,2025-02-06 21:30:29.918903+00:00,1
19967,CT,"Land Use, Land-Use Change and Forestry",Wetlands,LULUCF Sector Net Total,None,Wetlands Remaining Wetlands,Peatlands,Offsite,None,None,...,0.00738,0.0083,0.006432,0.005987,0.005705,0.005297,0.005545,usepa,2025-02-06 21:30:29.918903+00:00,1
19968,KS,"Land Use, Land-Use Change and Forestry",Wetlands,LULUCF Sector Net Total,None,Wetlands Remaining Wetlands,Coastal Wetlands Remaining Coastal Wetlands,Open Converted to Vegetated,None,None,...,0.00000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,usepa,2025-02-06 21:30:29.918903+00:00,1
19969,MA,"Land Use, Land-Use Change and Forestry",Wetlands,LULUCF Sector Net Total,None,Wetlands Remaining Wetlands,Peatlands,Onsite,None,None,...,0.00000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,usepa,2025-02-06 21:30:29.918903+00:00,1


Remove Duplicates and Keep Only Rank 1

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE us_climate_int.State_GHG_Emissions_Deduplicated AS
WITH ranked_data AS (
    SELECT *,
        ROW_NUMBER() OVER (
            PARTITION BY geo_ref, ghg, sector, subsector, econ_sector, econ_subsector, sub_category_1, fuel
            ORDER BY geo_ref DESC
        ) AS rank
    FROM us_climate_int.State_GHG_Emissions
)
SELECT * EXCEPT(rank)
FROM ranked_data
WHERE rank = 1;

Query is running:   0%|          |

""


Replace the Old Table

In [ ]:
%%bigquery
DROP TABLE us_climate_int.State_GHG_Emissions;

Query is running:   0%|          |

""


In [ ]:
%%bigquery
ALTER TABLE us_climate_int.State_GHG_Emissions_Deduplicated
RENAME TO State_GHG_Emissions;

Query is running:   0%|          |

""


Verify That Duplicates Are Removed

In [ ]:
%%bigquery
SELECT geo_ref, ghg, sector, subsector, econ_sector, econ_subsector, sub_category_1, fuel, COUNT(*) AS count
FROM us_climate_int.State_GHG_Emissions
GROUP BY geo_ref, ghg, sector, subsector, econ_sector, econ_subsector, sub_category_1, fuel
HAVING COUNT(*) > 1;

Query is running:   0%|          |

Downloading: |          |

,geo_ref,ghg,sector,subsector,econ_sector,econ_subsector,sub_category_1,fuel,count


Fixing name

In [ ]:
%%bigquery
create or replace table us_climate_int.State_GHG_Emission as
  select *
  from us_climate_int.State_GHG_Emissions

Query is running:   0%|          |

""


Clean up

In [ ]:
%%bigquery
drop table if exists us_climate_int.State_GHG_Emissions

Query is running:   0%|          |

""


## Key Constraints
#### Primary Keys

In [ ]:
%%bigquery
SELECT table_name, constraint_name
FROM `us_climate_int.INFORMATION_SCHEMA.TABLE_CONSTRAINTS`
WHERE constraint_type = 'PRIMARY KEY';

Query is running:   0%|          |

Downloading:   0%|          |

,table_name,constraint_name
0,Carbon_Capture_Categories,Carbon_Capture_Categories.pk$
1,State_Average_Temperature,State_Average_Temperature.pk$
2,Carbon_Capture_Facility,Carbon_Capture_Facility.pk$
3,Climate_Risk_Projection,Climate_Risk_Projection.pk$
4,Policies,Policies.pk$
5,Geo_References,Geo_References.pk$


In [ ]:
%%bigquery
ALTER TABLE us_climate_int.State_Climate_Policy ADD PRIMARY KEY (state, policy) NOT ENFORCED;
ALTER TABLE us_climate_int.State_Disaster ADD PRIMARY KEY (state, year) NOT ENFORCED;
ALTER TABLE us_climate_int.State_Electricity_Generation_Fuel_Share ADD PRIMARY KEY (state) NOT ENFORCED;
ALTER TABLE us_climate_int.State_GHG_Emission ADD PRIMARY KEY (geo_ref, econ_sector, econ_subsector, sector, subsector, category, sub_category_1, fuel, ghg) NOT ENFORCED;
ALTER TABLE us_climate_int.Facility_GHG_Emission ADD PRIMARY KEY (facility_id, year) NOT ENFORCED;
ALTER TABLE us_climate_int.Organizations ADD PRIMARY KEY (organization_name) NOT ENFORCED;

Query is running:   0%|          |

""


#### Foreign Keys

In [ ]:
%%bigquery
SELECT table_name, column_name
FROM `us_climate_int.INFORMATION_SCHEMA.KEY_COLUMN_USAGE`
WHERE constraint_name LIKE '%pk%';

Query is running:   0%|          |

Downloading:   0%|          |

,table_name,column_name
0,Organizations,organization_name
1,State_GHG_Emission,geo_ref
2,State_GHG_Emission,econ_sector
3,State_GHG_Emission,econ_subsector
4,State_GHG_Emission,sector
5,State_GHG_Emission,subsector
6,State_GHG_Emission,category
7,State_GHG_Emission,sub_category_1
8,State_GHG_Emission,fuel
9,State_GHG_Emission,ghg


In [ ]:
%%bigquery
ALTER TABLE us_climate_int.Organizations
ADD CONSTRAINT organizations_fk_organization_id
FOREIGN KEY (organization_name)
REFERENCES us_climate_int.Carbon_Capture_Facility (id) NOT ENFORCED;

ALTER TABLE us_climate_int.Organizations
ADD CONSTRAINT organizations_fk_facility_id
FOREIGN KEY (organization_name)
REFERENCES us_climate_int.Facility_GHG_Emission (facility_id) NOT ENFORCED;

ALTER TABLE us_climate_int.Policies
ADD CONSTRAINT policy_fk_policy
FOREIGN KEY (policy)
REFERENCES us_climate_int.State_Climate_Policy (policy) NOT ENFORCED;

ALTER TABLE us_climate_int.Geo_References
ADD CONSTRAINT geo_id_fk_state
FOREIGN KEY (geo_id)
REFERENCES us_climate_int.State_Average_Temperature (state) NOT ENFORCED;

ALTER TABLE us_climate_int.Geo_References
ADD CONSTRAINT geo_id_fk_state_climate_policy
FOREIGN KEY (geo_id)
REFERENCES us_climate_int.State_Climate_Policy (state) NOT ENFORCED;

ALTER TABLE us_climate_int.Geo_References
ADD CONSTRAINT geo_id_fk_state_disaster
FOREIGN KEY (geo_id)
REFERENCES us_climate_int.State_Disaster (state) NOT ENFORCED;

ALTER TABLE us_climate_int.Geo_References
ADD CONSTRAINT geo_id_fk_state_ghg
FOREIGN KEY (geo_id)
REFERENCES us_climate_int.State_GHG_Emission (geo_ref) NOT ENFORCED;

ALTER TABLE us_climate_int.Facility_GHG_Emission
ADD CONSTRAINT facility_ghg_fk_state
FOREIGN KEY (facility_id)
REFERENCES us_climate_int.State_Electricity_Generation_Fuel_Share (state) NOT ENFORCED;


Executing query with job ID: 4d5f37d7-1003-478a-bcb7-d80a662fa509
Query executing: 0.20s


ERROR:
 400 GET https://bigquery.googleapis.com/bigquery/v2/projects/kiaraerica/queries/4d5f37d7-1003-478a-bcb7-d80a662fa509?maxResults=0&location=us-central1&prettyPrint=false: Query error: Referenced column id from us_climate_int.Carbon_Capture_Facility is not compatible with the referencing column organization_name at [4:52]

Location: us-central1
Job ID: 4d5f37d7-1003-478a-bcb7-d80a662fa509

